In [3]:
# data structure and working with data api's
import numpy as np
import pandas as pd
# graphing/plotting super cool charts api
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# trade api
import alpaca_trade_api as trader
# stock data api
import yfinance as yf
# processing data/getting indicators api
import talib

In [4]:
APCA_API_BASE_URL = 'https://paper-api.alpaca.markets'
APCA_API_KEY_ID = '**********'
ACPA_API_SECRET_KEY = '**************'


tapi = trader.REST(APCA_API_KEY_ID, ACPA_API_SECRET_KEY,base_url=APCA_API_BASE_URL, api_version='v2')

In [5]:
account = tapi.get_account()
account

Account({   'account_blocked': False,
    'account_number': 'PA2MAFFBO1WK',
    'buying_power': '400000',
    'cash': '100000',
    'created_at': '2020-05-26T02:43:38.960856Z',
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '400000',
    'equity': '100000',
    'id': '42eb0e5c-a534-495e-8197-9843cd30f217',
    'initial_margin': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '4',
    'pattern_day_trader': False,
    'portfolio_value': '100000',
    'regt_buying_power': '200000',
    'short_market_value': '0',
    'shorting_enabled': False,
    'sma': '0',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})

In [27]:
class Stoinks:    
    def __init__(self, tickers,indicators=['Averages20', 'BollingerBands', 'RSI'], history=('1d', '1m')):
        # when you make a new stoinks object, it'll automatically just do all this to initialize the object
        # to see method explanations go to those methods, self.Tickers = tickers just is initializing that variable,
        # same with self.Indicators
        self.Tickers = tickers
        self.Indicators = indicators
        self.parse_data()
        self.histories(period=history[0], interval=history[1])
        self.optimize()
        
    def parse_data(self):
        # fills up the dictionary stocks, which has the ticker objects that you access using the symbol,
        # eg: self.Stocks['aapl'] gives the ticker object for apple (ticker is the object type used by yfinance)
        self.Stocks = {}
        for symbol in self.Tickers:
            self.Stocks[symbol] = yf.Ticker(symbol)
    def clean_one(self, df, remove_last=True):
        # handy and quick function that just drops the Dividends and Stock Splits columns in all histories,
        # also gets rid of the last minute if remove_last is true
        if 'Dividends' in df and 'Stock Splits' in df:
            df.drop(labels=['Dividends', 'Stock Splits'], axis=1, inplace=True)
        if remove_last:
            df = df[:-1]
        return df
    def histories(self, period='1d', interval='1m'):
        # fills up the dictionary object histories, which has dataframes that you access using the symbol,
        # eg: self.Histories['msft'] gives the microsoft stock dataframe for the specified period and interval during initialization
        # also calls clean_one method to clean up the dataframe, with remove_last=True only if the interval is 1m or 2m
        self.Histories = {}
        for symbol in self.Tickers:
            self.Histories[symbol] = self.Stocks[symbol].history(period=period, interval=interval)
            if interval == '1m' or interval == '2m':
                self.Histories[symbol] = self.clean_one(self.Histories[symbol])
            else:
                self.Histories[symbol] = self.clean_one(self.Histories[symbol], remove_last=False)
    def get_buy_sell(self, diff1, diff2=0):
        try:
            if diff2 == 0:
                diff2 = diff1
        except ValueError:
            pass
        buy_times = []
        sell_times = []
        for i in range(1, len(diff1)):
            if (diff1[i-1] <= 0 and diff1[i] >= 0):
                buy_times.append(diff1.index[i])
            elif (diff2[i-1] >= 0 and diff2[i] <= 0):
                sell_times.append(diff2.index[i])
        return buy_times, sell_times
    
    
    
    def Averages(self, ticker, period1=20, period2=50, plot=True):
        # ok so looks very complicated, but I will try to simplify for myself:
        # close gets the Close data for the specified ticker.
        # From there, we get two SMAs that default to 20-period and 50-period, but you can specify if you want.
        # Taking the difference of the two SMAs can tell us when to buy and sell -
        # we call get_buy_sell, which iterates through the differences, and when the 20-p goes above 50-p it's a buy signal,
        # and vice versa when 20-p goes below 50-p.
        # Plotting is interesting: we make multiple traces with different plots, and then overlay the plots, leading to complete graph that is also interactive.
        df = self.Histories[ticker]
        close = df.Close
        sma1 = talib.SMA(close, timeperiod=period1)
        sma2 = talib.SMA(close, timeperiod=period2)
        diff = sma1 - sma2
        buy_times, sell_times = self.get_buy_sell(diff)
        buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close)
        if plot:
            trace1 = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name=str(ticker)+' Stock Data')
            trace4 = go.Scatter(x=sma1.index, y=sma1, name=str(period1)+' Period Moving Average')
            trace5 = go.Scatter(x=sma2.index, y=sma2, name=str(period2)+' Period Moving Average')
            trace2 = go.Scatter(x=buy_times.index, y=buy_times, mode='markers', name='Buy Points', marker_color='green', marker_size=10)
            trace3 = go.Scatter(x=sell_times.index, y=sell_times, mode='markers', name='Sell Points', marker_color='red', marker_size=10)
            fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5])
            fig.update_layout(xaxis_rangeslider_visible=False)
            fig.show()
        return buy_times, sell_times, close[sma1 > sma2].rename('Good'), close[sma1 <= sma2].rename('Bad')
    def Bollingers(self, ticker, period=20, plot=True):
        df = self.Histories[ticker]
        close = df.Close
        upper, middle, lower = talib.BBANDS(close, timeperiod=period)
        up_mid, middle, low_mid = talib.BBANDS(close, timeperiod=period, nbdevup=1, nbdevdn=1)
        diff1 = close - up_mid
        diff2 = close - low_mid
        buy_times, sell_times = self.get_buy_sell(diff1, diff2)
        buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close)
        if plot:
            trace1 = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name=str(ticker)+' Stock Data')
            trace2 = go.Scatter(x=middle.index, y=upper, line={'color':'rgba(0, 255, 0, 0.8)'}, name='Buy Range', showlegend=False)
            trace3 = go.Scatter(x=middle.index, y=up_mid, fill='tonexty', fillcolor='rgba(0, 255, 0, 0.2)', line={'color':'rgba(255, 255, 0, 0.8)'}, name='No Man\'s Land')
            trace4 = go.Scatter(x=middle.index, y=low_mid, fill='tonexty', fillcolor='rgba(255, 255, 0, 0.2)', line={'color':'rgba(255, 255, 0, 0.8)'}, name='No Man\'s Land')
            trace5 = go.Scatter(x=middle.index, y=lower, fill='tonexty', fillcolor='rgba(255, 0, 0, 0.2)', line={'color':'rgba(100, 0, 0, 0.8)'}, name='Sell Range')
            trace6 = go.Scatter(x=buy_times.index, y=buy_times, mode='markers', name='Buy Points', marker_color='green', marker_size=10)
            trace7 = go.Scatter(x=sell_times.index, y=sell_times, mode='markers', name='Sell Points', marker_color='red', marker_size=10)
            fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5, trace6, trace7])
            fig.update_layout(xaxis_rangeslider_visible=False)
            fig.show()
        return buy_times, sell_times, close[close > up_mid].rename('Good'), close[close <= low_mid].rename('Bad')
    def RSI(self, ticker, period=14, sell_at=None, buy_at=None, plot=True):
        df = self.Histories[ticker]
        close = df.Close
        rsi = talib.RSI(close, timeperiod=period)
        sell_point = rsi.max() - ( rsi.std() / 2 ) or sell_at
        buy_point = rsi.min() + ( rsi.std() / 2 ) or buy_at
        diff1 = rsi - sell_point
        diff2 = rsi - buy_point
        sell_times, buy_times = self.get_buy_sell(diff1, diff2)
        buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close)
        if plot:
            sns.set()
            fig = make_subplots(rows=2, cols=1, column_widths=[1], row_heights=[0.8, 0.2], specs =
                               [ [ {'type': 'candlestick'} ], [ {'type': 'scatter'} ] ], shared_xaxes=True )
            trace1 = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name=str(ticker)+' Stock Data')
            trace2 = go.Scatter(x=buy_times.index, y=buy_times, mode='markers', name='Buy Points', marker_color='green', marker_size=10)
            trace3 = go.Scatter(x=sell_times.index, y=sell_times, mode='markers', name='Sell Points', marker_color='red', marker_size=10)
            trace4 = go.Scatter(x=rsi.index, y=rsi, name='RSI')
            fig.add_trace(trace1, row=1, col=1)
            fig.add_trace(trace2, row=1, col=1)
            fig.add_trace(trace3, row=1, col=1)
            fig.add_trace(trace4, row=2, col=1)
            fig.update_layout(xaxis_rangeslider_visible=False)
            fig.show()
        return buy_times, sell_times, None, None
    def MACD(self):
        return None
    
    
    
    
    def indicator(self, ticker, indicator, plot=False):
        if indicator == 'Averages20':
            return self.Averages(ticker, plot=plot)
        if indicator == 'Averages60':
            return self.Averages(ticker, period1=60, period2=90, plot=plot)
        if indicator == 'Averages50':
            return self.Averages(ticker, period1=50, period2=200, plot=plot)
        if indicator == 'BollingerBands':
            return self.Bollingers(ticker, plot=plot)
        if indicator == 'RSI':
            return self.RSI(ticker, plot=plot)
        
    def clean_two(self, buy, sell):
        df = pd.concat([buy.to_frame(), sell.to_frame()])
        df.sort_values('Date/Time', inplace=True)
        try:
            if np.isnan(df.Buy[df.index[0]]):
                df.drop(labels=df.index[0], axis=0, inplace=True)
            if np.isnan(df.Sell[df.index[-1]]):
                df.drop(labels=df.index[-1], axis=0, inplace=True)
        except IndexError:
            pass
        return df
    
    def parse_buy_sell(self, buy, sell, close):
        df = pd.concat([close[buy].rename('Buy', inplace=True).to_frame(), close[sell].rename('Sell', inplace=True).to_frame()])
        df.index.name = 'Date/Time'
        df.sort_values('Date/Time', inplace=True)
        current=''
        for index in df.index:
            if np.isnan(df.Buy[index]):
                if current == 'Sell':
                    df.drop(labels=index, axis=0, inplace=True)
                else:
                    current = 'Sell'
            else:
                if current == 'Buy':
                    df.drop(labels=index, axis=0, inplace=True)
                else:
                    current = 'Buy'
        buy = df['Buy']
        sell = df['Sell']
        buy.dropna(axis=0, inplace=True)
        sell.dropna(axis=0, inplace=True)
        return buy, sell
        
    def cost_score(self, ticker, indicator):
        buy, sell, dummy, dummy = self.indicator(ticker, indicator)
        df = self.clean_two(buy, sell)
        profits = []
        for i, index in enumerate(df.index):
            if i%2 == 0:
                buy = df.Buy[index]
            else:
                sell = df.Sell[index]
                profit = sell - buy
                profits.append(profit)
        profits = np.array(profits)
        return profits.mean(), profits
    
    def score_all(self):
        self.Scores = {}
        for ticker in self.Tickers:
            test = {}
            for indicator in self.Indicators:
                test[indicator] = self.cost_score(ticker, indicator)[0]
            self.Scores[ticker] = test
        self.Scores = pd.DataFrame(self.Scores)
        return self.Scores
    
    def optimize(self):
        self.score_all()
        self.Optimal = {}
        for ticker in self.Scores:
            self.Optimal[ticker] = self.Scores[ticker].idxmax()
        return self.Optimal
    

In [12]:
#pls_work = [{"ticker":"A","name":"Agilent Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AA","name":"Alcoa Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AAC","name":"AAC Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AAN","name":"Aaron&#39;s,  Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AAP","name":"Advance Auto Parts Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AAT","name":"American Assets Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AB","name":"AllianceBernstein Holding L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ABB","name":"ABB Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ABBV","name":"AbbVie Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ABC","name":"AmerisourceBergen Corporation (Holding Co)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ABEV","name":"Ambev S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ABG","name":"Asbury Automotive Group Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ABM","name":"ABM Industries Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ABR","name":"Arbor Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ABR^A","name":"Arbor Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ABR^B","name":"Arbor Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ABR^C","name":"Arbor Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ABT","name":"Abbott Laboratories","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AC","name":"Associated Capital Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ACA","name":"Arcosa, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ACB","name":"Aurora Cannabis Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ACC","name":"American Campus Communities Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ACCO","name":"Acco Brands Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ACH","name":"Aluminum Corporation of China Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ACM","name":"AECOM","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ACN","name":"Accenture plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ACP","name":"Aberdeen Income Credit Strategies Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ACRE","name":"Ares Commercial Real Estate Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ACV","name":"AllianzGI Diversified Income & Convertible Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ADC","name":"Agree Realty Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ADM","name":"Archer-Daniels-Midland Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ADNT","name":"Adient plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ADS","name":"Alliance Data Systems Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ADSW","name":"Advanced Disposal Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ADT","name":"ADT Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ADX","name":"Adams Diversified Equity Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AEB","name":"Aegon NV","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AEE","name":"Ameren Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AEG","name":"Aegon NV","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AEH","name":"Aegon NV","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AEL","name":"American Equity Investment Life Holding Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AEM","name":"Agnico Eagle Mines Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AEO","name":"American Eagle Outfitters, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AEP","name":"American Electric Power Company, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AEP^B","name":"American Electric Power Company, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AER","name":"Aercap Holdings N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AES","name":"The AES Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AFB","name":"Alliance National Municipal Income Fund Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AFC","name":"Ares Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AFG","name":"American Financial Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AFGB","name":"American Financial Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AFGE","name":"American Financial Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AFGH","name":"American Financial Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AFI","name":"Armstrong Flooring, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AFL","name":"Aflac Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AFT","name":"Apollo Senior Floating Rate Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AG","name":"First Majestic Silver Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGCO","name":"AGCO Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGD","name":"Aberdeen Global Dynamic Dividend Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGI","name":"Alamos Gold Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGM","name":"Federal Agricultural Mortgage Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGM^A","name":"Federal Agricultural Mortgage Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGM.A","name":"Federal Agricultural Mortgage Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGM^C","name":"Federal Agricultural Mortgage Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGM^D","name":"Federal Agricultural Mortgage Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGN","name":"Allergan plc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGO","name":"Assured Guaranty Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGO^B","name":"Assured Guaranty Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGO^E","name":"Assured Guaranty Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGO^F","name":"Assured Guaranty Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGR","name":"Avangrid, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGRO","name":"Adecoagro S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGS","name":"PlayAGS, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AGX","name":"Argan, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AHC","name":"A.H. Belo Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AHH","name":"Armada Hoffler Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AHH^A","name":"Armada Hoffler Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AHL^C","name":"Aspen Insurance Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AHL^D","name":"Aspen Insurance Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AHL^E","name":"Aspen Insurance Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AHT","name":"Ashford Hospitality Trust Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AHT^D","name":"Ashford Hospitality Trust Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AHT^F","name":"Ashford Hospitality Trust Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AHT^G","name":"Ashford Hospitality Trust Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AHT^H","name":"Ashford Hospitality Trust Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AHT^I","name":"Ashford Hospitality Trust Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AI","name":"Arlington Asset Investment Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AI^B","name":"Arlington Asset Investment Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AIC","name":"Arlington Asset Investment Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AI^C","name":"Arlington Asset Investment Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AIF","name":"Apollo Tactical Income Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AIG","name":"American International Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AIG^A","name":"American International Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AIG.WS","name":"American International Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AIN","name":"Albany International Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AIR","name":"AAR Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AIT","name":"Applied Industrial Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AIV","name":"Apartment Investment and Management Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AIW","name":"Arlington Asset Investment Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AIZ","name":"Assurant, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AIZP","name":"Assurant, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AJG","name":"Arthur J. Gallagher & Co.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AJRD","name":"Aerojet Rocketdyne Holdings, Inc. ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AJX","name":"Great Ajax Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AJXA","name":"Great Ajax Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AKO.A","name":"Embotelladora Andina S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AKO.B","name":"Embotelladora Andina S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AKR","name":"Acadia Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AKS","name":"AK Steel Holding Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AL","name":"Air Lease Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AL^A","name":"Air Lease Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALB","name":"Albemarle Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALC","name":"Alcon Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALE","name":"Allete, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALEX","name":"Alexander & Baldwin, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALG","name":"Alamo Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALK","name":"Alaska Air Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALL","name":"Allstate Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALL^A","name":"Allstate Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALL^B","name":"Allstate Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALL^D.CL","name":"Allstate Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALLE","name":"Allegion plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALL^E.CL","name":"Allstate Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALL^F.CL","name":"Allstate Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALL^G","name":"Allstate Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALL^H","name":"Allstate Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALLY","name":"Ally Financial Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALLY^A","name":"Ally Financial Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALP^Q","name":"Alabama Power Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALSN","name":"Allison Transmission Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALV","name":"Autoliv, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ALX","name":"Alexander&#39;s, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AM","name":"Antero Midstream Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMC","name":"AMC Entertainment Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMCR","name":"Amcor plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AME","name":"AMTEK, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMG","name":"Affiliated Managers Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMH","name":"American Homes 4 Rent","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMH^D","name":"American Homes 4 Rent","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMH^E","name":"American Homes 4 Rent","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMH^F","name":"American Homes 4 Rent","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMH^G","name":"American Homes 4 Rent","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMH^H","name":"American Homes 4 Rent","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMK","name":"AssetMark Financial Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMN","name":"AMN Healthcare Services Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMOV","name":"America Movil, S.A.B. de C.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMP","name":"AMERIPRISE FINANCIAL SERVICES, INC.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMPY","name":"MIDSTATES PETROLEUM COMPANY, INC.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMRC","name":"Ameresco, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMRX","name":"Amneal Pharmaceuticals, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMT","name":"American Tower Corporation (REIT)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AMX","name":"America Movil, S.A.B. de C.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AN","name":"AutoNation, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ANET","name":"Arista Networks, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ANF","name":"Abercrombie & Fitch Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ANFI","name":"Amira Nature Foods Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ANH","name":"Anworth Mortgage Asset  Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ANH^A","name":"Anworth Mortgage Asset  Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ANH^B","name":"Anworth Mortgage Asset  Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ANH^C","name":"Anworth Mortgage Asset  Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ANTM","name":"Anthem, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AOD","name":"Aberdeen Total Dynamic Dividend Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AON","name":"Aon plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AOS","name":"A.O Smith Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AP","name":"Ampco-Pittsburgh Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"APA","name":"Apache Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"APAM","name":"Artisan Partners Asset Management Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"APD","name":"Air Products and Chemicals, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"APH","name":"Amphenol Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"APHA","name":"Aphria Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"APLE","name":"Apple Hospitality REIT, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"APO","name":"Apollo Global Management, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"APO^A","name":"Apollo Global Management, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"APO^B","name":"Apollo Global Management, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"APRN","name":"Blue Apron Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"APTS","name":"Preferred Apartment Communities, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"APTV","name":"Aptiv PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"APY","name":"Apergy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AQ","name":"Aquantia Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AQN","name":"Algonquin Power & Utilities Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AQNA","name":"Algonquin Power & Utilities Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AQNB","name":"Algonquin Power & Utilities Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AQUA","name":"Evoqua Water Technologies Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AR","name":"Antero Resources Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARA","name":"American Renal Associates Holdings, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARC","name":"ARC Document Solutions, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARCH","name":"Arch Coal, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARCO","name":"Arcos Dorados Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARD","name":"Ardagh Group S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARDC","name":"Ares Dynamic Credit Allocation Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARE","name":"Alexandria Real Estate Equities, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARE^D","name":"Alexandria Real Estate Equities, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARES","name":"Ares Management Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARES^A","name":"Ares Management Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARGD","name":"Argo Group International Holdings, Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARGO","name":"Argo Group International Holdings, Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARI","name":"Apollo Commercial Real Estate Finance","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARL","name":"American Realty Investors, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARLO","name":"Arlo Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARMK","name":"Aramark","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARNC","name":"Arconic Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AROC","name":"Archrock, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARR","name":"ARMOUR Residential REIT, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARR^B","name":"ARMOUR Residential REIT, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ARW","name":"Arrow Electronics, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ASA","name":"ASA Gold and Precious Metals Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ASB","name":"Associated Banc-Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ASB^C","name":"Associated Banc-Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ASB^D","name":"Associated Banc-Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ASB^E","name":"Associated Banc-Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ASC","name":"Ardmore Shipping Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ASG","name":"Liberty All-Star Growth Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ASGN","name":"ASGN Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ASH","name":"Ashland Global Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ASIX","name":"AdvanSix Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ASPN","name":"Aspen Aerogels, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ASR","name":"Grupo Aeroportuario del Sureste, S.A. de C.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ASX","name":"ASE Technology Holding Co., Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AT","name":"Atlantic Power Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ATEN","name":"A10 Networks, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ATGE","name":"Adtalem Global Education Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ATH","name":"Athene Holding Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ATH^A","name":"Athene Holding Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ATHM","name":"Autohome Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ATI","name":"Allegheny Technologies Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ATKR","name":"Atkore International Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ATO","name":"Atmos Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ATR","name":"AptarGroup, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ATTO","name":"Atento S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ATU","name":"Actuant Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ATUS","name":"Altice USA, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ATV","name":"Acorn International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AU","name":"AngloGold Ashanti Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AUO","name":"AU Optronics Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AUY","name":"Yamana Gold Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVA","name":"Avista Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVAL","name":"Grupo Aval Acciones y Valores S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVB","name":"AvalonBay Communities, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVD","name":"American Vanguard Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVH","name":"Avianca Holdings S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVK","name":"Advent Convertible and Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVLR","name":"Avalara, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVNS","name":"Avanos Medical, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVP","name":"Avon Products, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVTR","name":"Avantor, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVTR^A","name":"Avantor, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVX","name":"AVX Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVY","name":"Avery Dennison Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AVYA","name":"Avaya Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AWF","name":"Alliance World Dollar Government Fund II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AWI","name":"Armstrong World Industries Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AWK","name":"American Water Works","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AWP","name":"Aberdeen Global Premier Properties Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AWR","name":"American States Water Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AX","name":"Axos Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AXE","name":"Anixter International Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AXL","name":"American Axle & Manufacturing Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AXO","name":"Axos Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AXP","name":"American Express Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AXR","name":"AMREP Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AXS","name":"Axis Capital Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AXS^D","name":"Axis Capital Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AXS^E","name":"Axis Capital Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AXTA","name":"Axalta Coating Systems Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AYI","name":"Acuity Brands, Inc. ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AYR","name":"Aircastle Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AYX","name":"Alteryx, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AZN","name":"Astrazeneca PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AZO","name":"AutoZone, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AZRE","name":"Azure Power Global Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AZUL","name":"Azul S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"AZZ","name":"AZZ Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"B","name":"Barnes Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BA","name":"Boeing Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BABA","name":"Alibaba Group Holding Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAC","name":"Bank of America Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAC^A","name":"Bank of America Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAC^B","name":"Bank of America Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAC^C","name":"Bank of America Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAC^E","name":"Bank of America Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAC^K","name":"Bank of America Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAC^L","name":"Bank of America Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAC^M","name":"Bank of America Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAC^Y","name":"Bank of America Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAF","name":"BlackRock Income Investment Quality Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAH","name":"Booz Allen Hamilton Holding Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAM","name":"Brookfield Asset Management Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BANC","name":"Banc of California, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BANC^D","name":"Banc of California, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BANC^E","name":"Banc of California, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAP","name":"Credicorp Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAS","name":"Basic Energy Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BAX","name":"Baxter International Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BB","name":"BlackBerry Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBAR","name":"BBVA Banco Frances S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBD","name":"Banco Bradesco Sa","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBDC","name":"Barings BDC, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBDO","name":"Banco Bradesco Sa","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBF","name":"BlackRock Municipal Income Investment Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBK","name":"Blackrock Municipal Bond Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBL","name":"BHP Group Plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBN","name":"BalckRock Taxable Municipal Bond Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBT","name":"BB&T Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBT^F","name":"BB&T Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBT^G","name":"BB&T Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBT^H","name":"BB&T Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBU","name":"Brookfield Business Partners L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBVA","name":"Banco Bilbao Viscaya Argentaria S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBW","name":"Build-A-Bear Workshop, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBX","name":"BBX Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BBY","name":"Best Buy Co., Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BC","name":"Brunswick Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BC^A","name":"Brunswick Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BC^B","name":"Brunswick Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BCC","name":"Boise Cascade, L.L.C.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BC^C","name":"Brunswick Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BCE","name":"BCE, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BCEI","name":"Bonanza Creek Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BCH","name":"Banco De Chile","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BCO","name":"Brink&#39;s Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BCRH","name":"Blue Capital Reinsurance Holdings Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BCS","name":"Barclays PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BCSF","name":"Bain Capital Specialty Finance, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BCX","name":"BlackRock Resources","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BDC","name":"Belden Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BDJ","name":"Blackrock Enhanced Equity Dividend Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BDN","name":"Brandywine Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BDX","name":"Becton, Dickinson and Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BDXA","name":"Becton, Dickinson and Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BE","name":"Bloom Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BEDU","name":"Bright Scholar Education Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BEN","name":"Franklin Resources, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BEP","name":"Brookfield Renewable Partners L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BERY","name":"Berry Global Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BEST","name":"BEST Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BF.A","name":"Brown Forman Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BFAM","name":"Bright Horizons Family Solutions Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BF.B","name":"Brown Forman Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BFK","name":"BlackRock Municipal Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BFO","name":"Blackrock Florida Municipal 2020 Term Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BFS","name":"Saul Centers, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BFS^C","name":"Saul Centers, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BFS^D","name":"Saul Centers, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BFY","name":"BlackRock New York Municipal Income Trust II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BFZ","name":"BlackRock California Municipal Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BG","name":"Bunge Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BGB","name":"Blackstone / GSO Strategic Credit Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BGG","name":"Briggs & Stratton Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BGH","name":"Babson Global Short Duration High Yield Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BGIO","name":"BlackRock 2022 Global Income Opportunity Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BGR","name":"BlackRock Energy and Resources Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BGS","name":"B&G Foods, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BGT","name":"Blackrock Global","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BGX","name":"Blackstone GSO Long Short Credit Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BGY","name":"BLACKROCK INTERNATIONAL, LTD.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BH","name":"Biglari Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BH.A","name":"Biglari Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BHC","name":"Bausch Health Companies Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BHE","name":"Benchmark Electronics, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BHGE","name":"Baker Hughes, a GE company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BHK","name":"Blackrock Core Bond Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BHLB","name":"Berkshire Hills Bancorp, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BHP","name":"BHP Group Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BHR","name":"Braemar Hotels & Resorts Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BHR^B","name":"Braemar Hotels & Resorts Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BHR^D","name":"Braemar Hotels & Resorts Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BHV","name":"BlackRock Virginia Municipal Bond Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BHVN","name":"Biohaven Pharmaceutical Holding Company Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BID","name":"Sotheby&#39;s","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BIF","name":"USLIFE Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BIG","name":"Big Lots, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BIO","name":"Bio-Rad Laboratories, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BIO.B","name":"Bio-Rad Laboratories, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BIP","name":"Brookfield Infrastructure Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BIT","name":"BlackRock Multi-Sector Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BITA","name":"Bitauto Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BJ","name":"BJ&#39;s Wholesale Club Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BK","name":"Bank Of New York Mellon Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BK^C","name":"Bank Of New York Mellon Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BKD","name":"Brookdale Senior Living Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BKE","name":"Buckle, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BKH","name":"Black Hills Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BKI","name":"Black Knight, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BKK","name":"Blackrock Municipal 2020 Term Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BKN","name":"BlackRock Investment Quality Municipal Trust Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BKT","name":"BlackRock Income Trust Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BKU","name":"BankUnited, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BLD","name":"TopBuild Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BLE","name":"BlackRock Municipal Income Trust II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BLK","name":"BlackRock, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BLL","name":"Ball Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BLW","name":"Citigroup Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BLX","name":"Banco Latinoamericano de Comercio Exterior, S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BMA","name":"Macro Bank Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BME","name":"Blackrock Health Sciences Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BMI","name":"Badger Meter, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BML^G","name":"Bank of America Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BML^H","name":"Bank of America Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BML^J","name":"Bank of America Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BML^L","name":"Bank of America Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BMO","name":"Bank Of Montreal","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BMY","name":"Bristol-Myers Squibb Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BNED","name":"Barnes & Noble Education, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BNS","name":"Bank of Nova Scotia (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BNY","name":"BlackRock New York Investment Quality Municipal Trust Inc. (Th","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BOE","name":"Blackrock Global","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BOH","name":"Bank of Hawaii Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BOOT","name":"Boot Barn Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BORR","name":"Borr Drilling Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BOX","name":"Box, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BP","name":"BP p.l.c.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BPL","name":"Buckeye Partners L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BPMP","name":"BP Midstream Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BPT","name":"BP Prudhoe Bay Royalty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BQH","name":"Blackrock New York Municipal Bond Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BR","name":"Broadridge Financial Solutions, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BRC","name":"Brady Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BRFS","name":"BRF S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BRK.A","name":"Berkshire Hathaway Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BRK.B","name":"Berkshire Hathaway Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BRO","name":"Brown & Brown, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BRPM","name":"B. Riley Principal Merger Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BRPM.U","name":"B. Riley Principal Merger Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BRPM.WS","name":"B. Riley Principal Merger Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BRT","name":"BRT Apartments Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BRX","name":"Brixmor Property Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BSA","name":"BrightSphere Investment Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BSAC","name":"Banco Santander Chile","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BSBR","name":"Banco Santander Brasil SA","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BSD","name":"BlackRock Strategic Municipal Trust Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BSE","name":"Blackrock New York Municipal Income Quality Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BSIG","name":"BrightSphere Investment Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BSL","name":"Blackstone GSO Senior Floating Rate Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BSM","name":"Black Stone Minerals, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BSMX","name":"Banco Santander Mexico, S.A., Institucion de Ban","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BST","name":"BlackRock Science and Technology Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BSTZ","name":"BlackRock Science and Technology Trust II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BSX","name":"Boston Scientific Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BT","name":"BT Group plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BTA","name":"BlackRock Long-Term Municipal Advantage Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BTE","name":"Baytex Energy Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BTI","name":"British American Tobacco p.l.c.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BTO","name":"John Hancock Financial Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BTT","name":"BlackRock Municipal Target Term Trust Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BTU","name":"Peabody Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BTZ","name":"BlackRock Credit Allocation Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BUD","name":"Anheuser-Busch Inbev SA","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BUI","name":"BlackRock Utility, Infrastructure & Power Opportun","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BURL","name":"Burlington Stores, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BV","name":"BrightView Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BVN","name":"Buenaventura Mining Company Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BW","name":"Babcock","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BWA","name":"BorgWarner Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BWG","name":"BrandywineGLOBAL Global Income Opportunities Fund ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BWXT","name":"BWX Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BX","name":"The Blackstone Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BXC","name":"BlueLinx Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BXG","name":"Bluegreen Vacations Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BXMT","name":"Capital Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BXMX","name":"Nuveen S&P 500 Buy-Write Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BXP","name":"Boston Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BXP^B","name":"Boston Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BXS","name":"BancorpSouth Bank","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BY","name":"Byline Bancorp, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BYD","name":"Boyd Gaming Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BYM","name":"Blackrock Municipal Income Quality Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BZH","name":"Beazer Homes USA, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"BZM","name":"BlackRock Maryland Municipal Bond Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"C","name":"Citigroup Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CAAP","name":"Corporacion America Airports SA","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CABO","name":"Cable One, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CACI","name":"CACI International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CADE","name":"Cadence Bancorporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CAE","name":"CAE Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CAF","name":"Morgan Stanley China A Share Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CAG","name":"ConAgra Brands, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CAH","name":"Cardinal Health, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CAI","name":"CAI International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CAI^A","name":"CAI International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CAI^B","name":"CAI International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CAJ","name":"Canon, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CAL","name":"Caleres, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CALX","name":"Calix, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CANG","name":"Cango Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CAPL","name":"CrossAmerica Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CARS","name":"Cars.com Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CAT","name":"Caterpillar, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CATO","name":"Cato Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CB","name":"Chubb Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBB","name":"Cincinnati Bell Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBB^B","name":"Cincinnati Bell Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBD","name":"Companhia Brasileira de Distribuicao","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBH","name":"AllianzGI Convertible & Income 2024 Target Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBL","name":"CBL & Associates Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBL^D","name":"CBL & Associates Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBL^E","name":"CBL & Associates Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBM","name":"Cambrex Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBO","name":"CBO (Listing Market - NYSE - Networks A/E)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBPX","name":"Continental Building Products, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBRE","name":"CBRE Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBS","name":"CBS Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBS.A","name":"CBS Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBT","name":"Cabot Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBU","name":"Community Bank System, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBX","name":"CBX (Listing Market NYSE Networks AE","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CBZ","name":"CBIZ, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CC","name":"Chemours Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCC","name":"Clarivate Analytics Plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCEP","name":"Coca-Cola European Partners plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCH","name":"Collier Creek Holdings","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCH.U","name":"Collier Creek Holdings","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCH.WS","name":"Collier Creek Holdings","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCI","name":"Crown Castle International Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCI^A","name":"Crown Castle International Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCJ","name":"Cameco Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCK","name":"Crown Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCL","name":"Carnival Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCM","name":"Concord Medical Services Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCO","name":"Clear Channel Outdoor Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCR","name":"CONSOL Coal Resources LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCS","name":"Century Communities, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCU","name":"Compania Cervecerias Unidas, S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCX","name":"Churchill Capital Corp II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCX.U","name":"Churchill Capital Corp II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCX.WS","name":"Churchill Capital Corp II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CCZ","name":"Comcast Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CDAY","name":"Ceridian HCM Holding Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CDE","name":"Coeur Mining, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CDR","name":"Cedar Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CDR^B","name":"Cedar Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CDR^C","name":"Cedar Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CE","name":"Celanese Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CEA","name":"China Eastern Airlines Corporation Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CEE","name":"The Central and Eastern Europe Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CEIX","name":"CNX Resources Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CEL","name":"Cellcom Israel, Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CELP","name":"Cypress Energy Partners, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CEM","name":"ClearBridge MLP and Midstream Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CEN","name":"Center Coast Brookfield MLP & Energy Infrastructur","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CEO","name":"CNOOC Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CEPU","name":"Central Puerto S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CEQP","name":"Crestwood Equity Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CEQP^","name":"Crestwood Equity Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CF","name":"CF Industries Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CFG","name":"Citizens Financial Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CFG^D","name":"Citizens Financial Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CFR","name":"Cullen/Frost Bankers, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CFR^A","name":"Cullen/Frost Bankers, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CFX","name":"Colfax Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CFXA","name":"Colfax Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CGA","name":"China Green Agriculture, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CGC","name":"Canopy Growth Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHA","name":"China Telecom Corp Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHAP","name":"Chaparral Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHCT","name":"Community Healthcare Trust Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHD","name":"Church & Dwight Company, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHE","name":"Chemed Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHGG","name":"Chegg, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHH","name":"Choice Hotels International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHK","name":"Chesapeake Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHK^D","name":"Chesapeake Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHKR","name":"Chesapeake Granite Wash Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHL","name":"China Mobile (Hong Kong) Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHMI","name":"Cherry Hill Mortgage Investment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHMI^A","name":"Cherry Hill Mortgage Investment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHMI^B","name":"Cherry Hill Mortgage Investment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHN","name":"China Fund, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHRA","name":"Charah Solutions, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHS","name":"Chico&#39;s FAS, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHSP","name":"Chesapeake Lodging Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHT","name":"Chunghwa Telecom Co., Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHU","name":"China Unicom (Hong Kong) Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CHWY","name":"Chewy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CI","name":"Cigna Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIA","name":"Citizens, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIB","name":"BanColombia S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIEN","name":"Ciena Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIF","name":"Colonial Intermediate High Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIG","name":"Comp En De Mn Cemig ADS","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIG.C","name":"Comp En De Mn Cemig ADS","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CII","name":"Blackrock Capital and Income Strategies Fund Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIM","name":"Chimera Investment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIM^A","name":"Chimera Investment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIM^B","name":"Chimera Investment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIM^C","name":"Chimera Investment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIM^D","name":"Chimera Investment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CINR","name":"Ciner Resources LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIO","name":"City Office REIT, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIO^A","name":"City Office REIT, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIR","name":"CIRCOR International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CISN","name":"Cision Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CIT","name":"CIT Group Inc (DEL)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CJ","name":"C&J Energy Services, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"C^J","name":"Citigroup Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"C^K","name":"Citigroup Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CKH","name":"SEACOR Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CL","name":"Colgate-Palmolive Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLB","name":"Core Laboratories N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLDR","name":"Cloudera, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLDT","name":"Chatham Lodging Trust (REIT)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLF","name":"Cleveland-Cliffs Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLGX","name":"CoreLogic, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLH","name":"Clean Harbors, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLI","name":"Mack-Cali Realty Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLNC","name":"Colony Credit Real Estate, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLNY","name":"Colony Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLNY^B","name":"Colony Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLNY^E","name":"Colony Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLNY^G","name":"Colony Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLNY^H","name":"Colony Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLNY^I","name":"Colony Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLNY^J","name":"Colony Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLPR","name":"Clipper Realty Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLR","name":"Continental Resources, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLS","name":"Celestica, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLW","name":"Clearwater Paper Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CLX","name":"Clorox Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CM","name":"Canadian Imperial Bank of Commerce","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMA","name":"Comerica Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMC","name":"Commercial Metals Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMCM","name":"Cheetah Mobile Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMD","name":"Cantel Medical Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMG","name":"Chipotle Mexican Grill, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMI","name":"Cummins Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMO","name":"Capstead Mortgage Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMO^E","name":"Capstead Mortgage Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMP","name":"Compass Minerals International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMRE","name":"Costamare Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMRE^B","name":"Costamare Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMRE^C","name":"Costamare Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMRE^D","name":"Costamare Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMRE^E","name":"Costamare Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMS","name":"CMS Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMSA","name":"CMS Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMS^B","name":"CMS Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMSC","name":"CMS Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMSD","name":"CMS Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CMU","name":"Colonial Municipal Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"C^N","name":"Citigroup Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNA","name":"CNA Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNC","name":"Centene Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNDT","name":"Conduent Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNF","name":"CNFinance Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNHI","name":"CNH Industrial N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNI","name":"Canadian National Railway Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNK","name":"Cinemark Holdings Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNNE","name":"Cannae Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNO","name":"CNO Financial Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNP","name":"CenterPoint Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNP^B","name":"CenterPoint Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNQ","name":"Canadian Natural Resources Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNR","name":"Cornerstone Building Brands, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNS","name":"Cohen & Steers Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNX","name":"CNX Resources Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CNXM","name":"CNX Midstream Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CO","name":"Global Cord Blood Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CODI","name":"Compass Diversified Holdings","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CODI^A","name":"Compass Diversified Holdings","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CODI^B","name":"Compass Diversified Holdings","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COE","name":"China Online Education Group","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COF","name":"Capital One Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COF^C","name":"Capital One Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COF^D","name":"Capital One Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COF^F","name":"Capital One Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COF^G","name":"Capital One Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COF^H","name":"Capital One Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COF^P","name":"Capital One Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COG","name":"Cabot Oil & Gas Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COLD","name":"Americold Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COO","name":"The Cooper Companies, Inc. ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COP","name":"ConocoPhillips","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COR","name":"CoreSite Realty Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CORR","name":"CorEnergy Infrastructure Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CORR^A","name":"CorEnergy Infrastructure Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COT","name":"Cott Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"COTY","name":"Coty Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CP","name":"Canadian Pacific Railway Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CPA","name":"Copa Holdings, S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CPAC","name":"Cementos Pacasmayo S.A.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CPB","name":"Campbell Soup Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CPE","name":"Callon Petroleum Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CPF","name":"CPB Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CPG","name":"Crescent Point Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CPK","name":"Chesapeake Utilities Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CPL","name":"CPFL Energia S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CPLG","name":"CorePoint Lodging Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CPRI","name":"Capri Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CPS","name":"Cooper-Standard Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CPT","name":"Camden Property Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CR","name":"Crane Co.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CRC","name":"California Resources Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CRCM","name":"Care.com, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CRD.A","name":"Crawford & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CRD.B","name":"Crawford & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CRH","name":"CRH PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CRI","name":"Carter&#39;s, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CRK","name":"Comstock Resources, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CRL","name":"Charles River Laboratories International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CRM","name":"Salesforce.com Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CRR","name":"Carbo Ceramics, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CRS","name":"Carpenter Technology Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CRT","name":"Cross Timbers Royalty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CRY","name":"CryoLife, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CS","name":"Credit Suisse Group","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"C^S","name":"Citigroup Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CSL","name":"Carlisle Companies Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CSLT","name":"Castlight Health, inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CSS","name":"CSS Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CSTM","name":"Constellium SE","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CSU","name":"Capital Senior Living Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CSV","name":"Carriage Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTAA","name":"Qwest Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTA^A","name":"E.I. du Pont de Nemours and Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTA^B","name":"E.I. du Pont de Nemours and Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTB","name":"Cooper Tire & Rubber Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTBB","name":"Qwest Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTDD","name":"Qwest Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTEST","name":"NYSE Test One","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTEST.E","name":"NYSE Test One","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTEST.G","name":"NYSE Test One","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTEST.L","name":"NYSE Test One","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTEST.O","name":"NYSE Test One","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTEST.S","name":"NYSE Test One","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTEST.V","name":"NYSE Test One","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTK","name":"CooTek (Cayman) Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTL","name":"CenturyLink, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTLT","name":"Catalent, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTR","name":"ClearBridge MLP and Midstream Total Return Fund In","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTRA","name":"Contura Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTS","name":"CTS Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTST","name":"CannTrust Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTT           ","name":"CatchMark Timber Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTV","name":"Qwest Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTVA","name":"Corteva, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTY","name":"Qwest Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CTZ","name":"Qwest Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CUB","name":"Cubic Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CUBE","name":"CubeSmart","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CUBI","name":"Customers Bancorp, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CUBI^C","name":"Customers Bancorp, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CUBI^D","name":"Customers Bancorp, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CUBI^E","name":"Customers Bancorp, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CUBI^F","name":"Customers Bancorp, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CUK","name":"Carnival Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CULP","name":"Culp, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CURO","name":"CURO Group Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CUZ","name":"Cousins Properties Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CVA","name":"Covanta Holding Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CVE","name":"Cenovus Energy Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CVEO","name":"Civeo Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CVI","name":"CVR Energy Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CVIA","name":"Covia Holdings Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CVNA","name":"Carvana Co.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CVS","name":"CVS Health Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CVX","name":"Chevron Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CW","name":"Curtiss-Wright Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CWEN","name":"Clearway Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CWEN.A","name":"Clearway Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CWH","name":"Camping World Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CWK","name":"Cushman & Wakefield plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CWT","name":"California Water  Service Group Holding","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CX","name":"Cemex S.A.B. de C.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CXE","name":"Colonial High Income Municipal Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CXH","name":"Colonial Investment Grade Municipal Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CXO","name":"Concho Resources Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CXP","name":"Columbia Property Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CXW","name":"CoreCivic, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CYD","name":"China Yuchai International Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CYH","name":"Community Health Systems, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"CZZ","name":"Cosan Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"D","name":"Dominion Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DAC","name":"Danaos Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DAL","name":"Delta Air Lines, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DAN","name":"Dana Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DAR","name":"Darling Ingredients Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DAVA","name":"Endava plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DB","name":"Deutsche Bank AG","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DBD","name":"Diebold Nixdorf Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DBI","name":"Designer Brands Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DBL","name":"DoubleLine Opportunistic Credit Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DCF","name":"BNY Mellon Alcentra Global Credit Income 2024 Target Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DCI","name":"Donaldson Company, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DCO","name":"Ducommun Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DCP","name":"DCP Midstream LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DCP^B","name":"DCP Midstream LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DCP^C","name":"DCP Midstream LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DCUE","name":"Dominion Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DD","name":"DuPont de Nemours, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DDD","name":"3D Systems Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DDF","name":"Delaware Investments Dividend & Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DDS","name":"Dillard&#39;s, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DDT","name":"Dillard&#39;s, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DE","name":"Deere & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DEA","name":"Easterly Government Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DECK","name":"Deckers Outdoor Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DEI","name":"Douglas Emmett, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DELL","name":"Dell Technologies Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DEO","name":"Diageo plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DESP","name":"Despegar.com, Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DEX","name":"Delaware Enhanced Global Dividend","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DF","name":"Dean Foods Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DFIN","name":"Donnelley Financial Solutions, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DFP","name":"Flaherty & Crumrine Dynamic Preferred and Income Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DFS","name":"Discover Financial Services","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DG","name":"Dollar General Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DGX","name":"Quest Diagnostics Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DHF","name":"BNY Mellon High Yield Strategies Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DHI","name":"D.R. Horton, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DHR","name":"Danaher Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DHR^A","name":"Danaher Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DHT","name":"DHT Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DHX","name":"DHI Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DIAX","name":"Nuveen Dow 30SM Dynamic Overwrite Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DIN","name":"Dine Brands Global, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DIS","name":"Walt Disney Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DK","name":"Delek US Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DKL","name":"Delek Logistics Partners, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DKS","name":"Dick&#39;s Sporting Goods Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DKT","name":"Deutsch Bk Contingent Cap Tr V","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DL","name":"China Distance Education Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DLB","name":"Dolby Laboratories","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DLNG","name":"Dynagas LNG Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DLNG^A","name":"Dynagas LNG Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DLNG^B","name":"Dynagas LNG Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DLPH","name":"Delphi Technologies PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DLR","name":"Digital Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DLR^C","name":"Digital Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DLR^G","name":"Digital Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DLR^I","name":"Digital Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DLR^J","name":"Digital Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DLR^K","name":"Digital Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DLX","name":"Deluxe Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DMB","name":"BNY Mellon Municipal Bond Infrastructure Fund, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DMO","name":"Western Asset Mortgage Defined Opportunity Fund Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DNI","name":"Dividend and Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DNOW","name":"NOW Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DNP","name":"Duff & Phelps Utilities Income, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DNR","name":"Denbury Resources Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DO","name":"Diamond Offshore Drilling, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DOC","name":"Physicians Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DOOR","name":"Masonite International Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DOV","name":"Dover Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DOW","name":"Dow Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DPG","name":"Duff & Phelps Global Utility Income Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DPLO","name":"Diplomat Pharmacy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DPZ","name":"Domino&#39;s Pizza Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DQ","name":"DAQO New Energy Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DRD","name":"DRDGOLD Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DRE","name":"Duke Realty Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DRH","name":"Diamondrock Hospitality Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DRI","name":"Darden Restaurants, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DRQ","name":"Dril-Quip, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DRUA","name":"Dominion Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DS","name":"Drive Shack Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DS^B","name":"Drive Shack Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DS^C","name":"Drive Shack Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DS^D","name":"Drive Shack Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DSE","name":"Duff & Phelps Select MLP and Midstream Energy Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DSL","name":"DoubleLine Income Solutions Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DSM","name":"BNY Mellon Strategic Municipal Bond Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DSSI","name":"Diamond S Shipping Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DSU","name":"Blackrock Debt Strategies Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DSX","name":"Diana Shipping inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DSX^B","name":"Diana Shipping inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DT","name":"Dynatrace, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DTE","name":"DTE Energy Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DTF","name":"Duff & Phelps Utilities Tax-Free Income, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DTJ","name":"DTE Energy Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DTLA^","name":"Brookfield DTLA Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DTQ","name":"DTE Energy Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DTV","name":"DTE Energy Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DTW","name":"DTE Energy Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DTY","name":"DTE Energy Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DUC","name":"Duff & Phelps Utility & Corporate Bond Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DUK","name":"Duke Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DUK^A","name":"Duke Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DUKB","name":"Duke Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DUKH","name":"Duke Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DVA","name":"DaVita Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DVD","name":"Dover Motorsports, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DVN","name":"Devon Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DX","name":"Dynex Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DX^A","name":"Dynex Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DXB","name":"Deutsche Bank AG","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DX^B","name":"Dynex Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DXC","name":"DXC Technology Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"DY","name":"Dycom Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"E","name":"ENI S.p.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EAB","name":"Entergy Arkansas, LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EAE","name":"Entergy Arkansas, LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EAF","name":"GrafTech International Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EAI","name":"Entergy Arkansas, LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EARN","name":"Ellington Residential Mortgage REIT","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EAT","name":"Brinker International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EB","name":"Eventbrite, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EBF","name":"Ennis, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EBR","name":"Centrais Electricas Brasileiras S.A.- Eletrobras","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EBR.B","name":"Centrais Electricas Brasileiras S.A.- Eletrobras","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EBS","name":"Emergent Biosolutions, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EC","name":"Ecopetrol S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ECA","name":"Encana Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ECC           ","name":"Eagle Point Credit Company Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ECCA","name":"Eagle Point Credit Company Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ECCB","name":"Eagle Point Credit Company Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ECCX","name":"Eagle Point Credit Company Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ECCY","name":"Eagle Point Credit Company Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ECL","name":"Ecolab Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ECOM          ","name":"ChannelAdvisor Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ECT","name":"ECA Marcellus Trust I","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ED","name":"Consolidated Edison Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EDD","name":"Morgan Stanley Emerging Markets Domestic Debt Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EDF","name":"Stone Harbor Emerging Markets Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EDI","name":"Stone Harbor Emerging Markets Total Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EDN","name":"Empresa Distribuidora Y Comercializadora Norte S.A. (Edenor)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EDU","name":"New Oriental Education & Technology Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EE","name":"El Paso Electric Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EEA","name":"European Equity Fund, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EEX","name":"Emerald Expositions Events, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EFC","name":"Ellington Financial Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EFF","name":"Eaton vance Floating-Rate Income Plus Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EFL","name":"Eaton Vance Floating-Rate 2022 Target Term Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EFR","name":"Eaton Vance Senior Floating-Rate Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EFT","name":"Eaton Vance Floating Rate Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EFX","name":"Equifax, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EGF","name":"Blackrock Enhanced Government Fund, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EGHT","name":"8x8 Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EGIF","name":"Eagle Growth and Income Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EGO","name":"Eldorado Gold Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EGP","name":"EastGroup Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EGY","name":"VAALCO Energy, Inc. ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EHC","name":"Encompass Health Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EHI","name":"Western Asset Global High Income Fund Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EHT","name":"Eaton Vance High Income 2021 Target Term Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EIC","name":"Eagle Point Income Company Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EIG","name":"Employers Holdings Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EIX","name":"Edison International","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EL","name":"Estee Lauder Companies, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ELAN","name":"Elanco Animal Health Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ELC","name":"Entergy Louisiana, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ELF","name":"e.l.f. Beauty, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ELJ","name":"Entergy Louisiana, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ELP","name":"Companhia Paranaense de Energia (COPEL)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ELS","name":"Equity Lifestyle Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ELU","name":"Entergy Louisiana, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ELVT","name":"Elevate Credit, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ELY","name":"Callaway Golf Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EMD","name":"Western Asset Emerging Markets Debt Fund Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EME","name":"EMCOR Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EMF","name":"Templeton Emerging Markets Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EMN","name":"Eastman Chemical Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EMO","name":"ClearBridge Energy Midstream Opportunity Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EMP","name":"Entergy Mississippi, LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EMR","name":"Emerson Electric Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENB","name":"Enbridge Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENBA","name":"Enbridge Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENBL","name":"Enable Midstream Partners, LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENIA","name":"Enel Americas S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENIC","name":"Enel Chile S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENJ","name":"Entergy New Orleans, LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENLC","name":"EnLink Midstream, LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENO","name":"Entergy New Orleans, LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENR","name":"Energizer Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENR^A","name":"Energizer Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENS","name":"Enersys","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENV","name":"Envestnet, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENVA","name":"Enova International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ENZ","name":"Enzo Biochem, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EOD","name":"Wells Fargo Global Dividend Opportunity Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EOG","name":"EOG Resources, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EOI","name":"Eaton Vance Enhance Equity Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EOS","name":"Eaton Vance Enhanced Equity Income Fund II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EOT","name":"Eaton Vance Municipal Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EPAM","name":"EPAM Systems, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EPC","name":"Energizer Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EP^C","name":"El Paso Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EPD","name":"Enterprise Products Partners L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EPR","name":"EPR Properties","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EPR^C","name":"EPR Properties","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EPR^E","name":"EPR Properties","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EPR^G","name":"EPR Properties","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EPRT","name":"Essential Properties Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EQC","name":"Equity Commonwealth","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EQC^D","name":"Equity Commonwealth","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EQH","name":"AXA Equitable Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EQM","name":"EQM Midstream Partners, LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EQNR","name":"Equinor ASA","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EQR","name":"Equity Residential","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EQS","name":"Equus Total Return, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EQT","name":"EQT Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ERA","name":"Era Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ERF","name":"Enerplus Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ERJ","name":"Embraer S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EROS","name":"Eros International PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ES","name":"Eversource Energy","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ESE","name":"ESCO Technologies Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ESI","name":"Element Solutions Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ESNT","name":"Essent Group Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ESRT","name":"Empire State Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ESS","name":"Essex Property Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ESTC","name":"Elastic N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ESTE","name":"Earthstone Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ET","name":"Energy Transfer L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETB","name":"Eaton Vance Tax-Managed Buy-Write Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETG","name":"Eaton Vance Tax-Advantaged Global Dividend Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETH","name":"Ethan Allen Interiors Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETI^","name":"Entergy Texas Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETJ","name":"Eaton Vance Risk-Managed Diversified Equity Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETM","name":"Entercom Communications Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETN","name":"Eaton Corporation, PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETO","name":"Eaton Vance Tax-Advantage Global Dividend Opp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETP^C","name":"Energy Transfer Operating, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETP^D","name":"Energy Transfer Operating, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETP^E","name":"Energy Transfer Operating, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETR","name":"Entergy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETRN","name":"Equitrans Midstream Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETV","name":"Eaton Vance Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETW","name":"Eaton Vance Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETX           ","name":"Eaton Vance Municipal Income 2028 Term Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ETY","name":"Eaton Vance Tax-Managed Diversified Equity Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EURN","name":"Euronav NV","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EV","name":"Eaton Vance Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EVA","name":"Enviva Partners, LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EVC","name":"Entravision Communications Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EVF","name":"Eaton Vance Senior Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EVG","name":"Eaton Vance Short Diversified Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EVH","name":"Evolent Health, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EVN","name":"Eaton Vance Municipal Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EVR","name":"Evercore Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EVRG","name":"Evergy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EVRI","name":"Everi Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EVT","name":"Eaton Vance Tax Advantaged Dividend Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EVTC","name":"Evertec, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EW","name":"Edwards Lifesciences Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EXC","name":"Exelon Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EXD","name":"Eaton Vance Tax-Managed Buy-Write Strategy Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EXG","name":"Eaton Vance Tax-Managed Global Diversified Equity Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EXK","name":"Endeavour Silver Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EXP","name":"Eagle Materials Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EXPR","name":"Express, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EXR","name":"Extra Space Storage Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EXTN","name":"Exterran Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"EZT","name":"Entergy Texas Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"F","name":"Ford Motor Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FAF","name":"First American Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FAM","name":"First Trust/Aberdeen Global Opportunity Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"F^B","name":"Ford Motor Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FBC","name":"Flagstar Bancorp, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FBHS","name":"Fortune Brands Home & Security, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FBK","name":"FB Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FBM","name":"Foundation Building Materials, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FBP","name":"First BanCorp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FC","name":"Franklin Covey Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FCAU","name":"Fiat Chrysler Automobiles N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FCF","name":"First Commonwealth Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FCN","name":"FTI Consulting, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FCPT","name":"Four Corners Property Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FCT","name":"First Trust Senior Floating Rate Income Fund II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FCX","name":"Freeport-McMoran, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FDEU","name":"First Trust Dynamic Europe Equity Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FDP","name":"Fresh Del Monte Produce, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FDS","name":"FactSet Research Systems Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FDX","name":"FedEx Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FE","name":"FirstEnergy Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FEDU","name":"Four Seasons Education (Cayman) Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FEI           ","name":"First Trust MLP and Energy Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FELP","name":"Foresight Energy LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FENG","name":"Phoenix New Media Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FEO","name":"First Trust/Aberdeen Emerging Opportunity Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FET","name":"Forum Energy Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FF","name":"FutureFuel Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FFA","name":"First Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FFC","name":"Flaherty & Crumrine Preferred and Income Securitie","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FFG","name":"FBL Financial Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FG","name":"FGL Holdings","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FGB","name":"First Trust Specialty Finance and Financial Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FGP","name":"Ferrellgas Partners, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FG.WS","name":"FGL Holdings","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FHN","name":"First Horizon National Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FHN^A","name":"First Horizon National Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FI","name":"Frank&#39;s International N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FICO","name":"Fair Isaac Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FIF","name":"First Trust Energy Infrastructure Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FII","name":"Federated Investors, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FINS","name":"Angel Oak Financial Strategies Income Term Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FIS","name":"Fidelity National Information Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FIT","name":"Fitbit, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FIV","name":"First Trust Senior Floating Rate 2022 Target Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FIX","name":"Comfort Systems USA, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FL","name":"Foot Locker, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FLC","name":"Flaherty & Crumrine Total Return Fund Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FLNG","name":"FLEX LNG Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FLO","name":"Flowers Foods, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FLOW","name":"SPX FLOW, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FLR","name":"Fluor Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FLS","name":"Flowserve Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FLT","name":"FleetCor Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FLY","name":"Fly Leasing Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FMC","name":"FMC Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FMN","name":"Federated Premier Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FMO","name":"Fiduciary/Claymore Energy Infrastructure Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FMS","name":"Fresenius Medical Care Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FMX","name":"Fomento Economico Mexicano S.A.B. de C.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FMY","name":"First Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FN","name":"Fabrinet","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FNB","name":"F.N.B. Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FNB^E","name":"F.N.B. Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FND","name":"Floor & Decor Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FNF","name":"Fidelity National Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FNV","name":"Franco-Nevada Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FOE","name":"Ferro Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FOF","name":"Cohen & Steers Closed-End Opportunity Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FOR","name":"Forestar Group Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FPAC","name":"Far Point Acquisition Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FPAC.U","name":"Far Point Acquisition Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FPAC.WS","name":"Far Point Acquisition Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FPF","name":"First Trust Intermediate Duration Preferred & Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FPH","name":"Five Point Holdings, LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FPI","name":"Farmland Partners Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FPI^B","name":"Farmland Partners Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FPL","name":"First Trust New Opportunities MLP & Energy Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FR","name":"First Industrial Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FRA","name":"Blackrock Floating Rate Income Strategies Fund Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FRAC","name":"Keane Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FRC","name":"FIRST REPUBLIC BANK","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FRC^D","name":"FIRST REPUBLIC BANK","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FRC^F","name":"FIRST REPUBLIC BANK","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FRC^G","name":"FIRST REPUBLIC BANK","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FRC^H","name":"FIRST REPUBLIC BANK","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FRC^I","name":"FIRST REPUBLIC BANK","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FRO","name":"Frontline Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FRT","name":"Federal Realty Investment Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FRT^C","name":"Federal Realty Investment Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FSB","name":"Franklin Financial Network, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FSD","name":"First Trust High Income Long Short Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FSK","name":"FS KKR Capital Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FSLY","name":"Fastly, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FSM","name":"Fortuna Silver Mines Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FSS","name":"Federal Signal Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FT","name":"Franklin Universal Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FTAI","name":"Fortress Transportation and Infrastructure Investors LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FTCH","name":"Farfetch Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FTI","name":"TechnipFMC plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FTK","name":"Flotek Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FTS","name":"Fortis Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FTSI","name":"FTS International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FTV","name":"Fortive Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FTV^A","name":"Fortive Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FUL","name":"H. B. Fuller Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FUN","name":"Cedar Fair, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"FVRR","name":"Fiverr International Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"G","name":"Genpact Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GAB","name":"Gabelli Equity Trust, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GAB^D","name":"Gabelli Equity Trust, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GAB^G","name":"Gabelli Equity Trust, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GAB^H","name":"Gabelli Equity Trust, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GAB^J","name":"Gabelli Equity Trust, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GAM","name":"General American Investors, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GAM^B","name":"General American Investors, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GATX","name":"GATX Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GBAB","name":"Guggenheim Taxable Municipal Managed Duration Trst","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GBL","name":"Gamco Investors, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GBX","name":"Greenbrier Companies, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GCAP","name":"GAIN Capital Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GCI","name":"TEGNA Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GCO","name":"Genesco Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GCP","name":"GCP Applied Technologies Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GCV","name":"Gabelli Convertible and Income Securities Fund, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GCV^B","name":"Gabelli Convertible and Income Securities Fund, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GD","name":"General Dynamics Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GDDY","name":"GoDaddy Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GDI","name":"Gardner Denver Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GDL","name":"The GDL Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GDL^C","name":"The GDL Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GDO","name":"Western Asset Global Corporate Defined Opportunity Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GDOT","name":"Green Dot Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GDV","name":"The Gabelli Dividend & Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GDV^A","name":"The Gabelli Dividend & Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GDV^D","name":"The Gabelli Dividend & Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GDV^G","name":"The Gabelli Dividend & Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GDV^H","name":"The Gabelli Dividend & Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GE","name":"General Electric Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GEF","name":"Greif Bros. Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GEF.B","name":"Greif Bros. Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GEL","name":"Genesis Energy, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GEN           ","name":"Genesis Healthcare, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GEO","name":"Geo Group Inc (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GER","name":"Goldman Sachs MLP Energy Renaissance Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GES","name":"Guess?, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GF","name":"New Germany Fund, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GFF","name":"Griffon Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GFI","name":"Gold Fields Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GFY","name":"Western Asset Variable Rate Strategic Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GGB","name":"Gerdau S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GGG","name":"Graco Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GGM","name":"Guggenheim Credit Allocation Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GGT","name":"Gabelli Multi-Media Trust Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GGT^B","name":"Gabelli Multi-Media Trust Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GGT^E","name":"Gabelli Multi-Media Trust Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GGZ","name":"Gabelli Global Small and Mid Cap Value Trust (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GGZ^A","name":"Gabelli Global Small and Mid Cap Value Trust (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GHC","name":"Graham Holdings Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GHG","name":"GreenTree Hospitality Group Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GHL","name":"Greenhill & Co., Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GHM","name":"Graham Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GHY","name":"PGIM Global High Yield Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GIB","name":"CGI Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GIG","name":"GigCapital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GIG~","name":"GigCapital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GIG.U","name":"GigCapital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GIG.WS","name":"GigCapital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GIL","name":"Gildan Activewear, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GIM","name":"Templeton Global Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GIS","name":"General Mills, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GIX","name":"GigCapital2, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GIX~","name":"GigCapital2, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GIX.U","name":"GigCapital2, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GIX.WS","name":"GigCapital2, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GJH","name":"STRATS Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GJO","name":"STRATS Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GJP","name":"Synthetic Fixed-Income Securities, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GJR","name":"Synthetic Fixed-Income Securities, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GJS","name":"STRATS Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GJT","name":"Synthetic Fixed-Income Securities, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GJV","name":"Synthetic Fixed-Income Securities, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GKOS","name":"Glaukos Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GL","name":"Globe Life Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GL^C","name":"Globe Life Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GLOB","name":"Globant S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GLOG","name":"GasLog LP.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GLOG^A","name":"GasLog LP.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GLOP","name":"GasLog Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GLOP^A","name":"GasLog Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GLOP^B","name":"GasLog Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GLOP^C","name":"GasLog Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GLP","name":"Global Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GLP^A","name":"Global Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GLT","name":"Glatfelter","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GLW","name":"Corning Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GM","name":"General Motors Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GME","name":"Gamestop Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GMED","name":"Globus Medical, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GMRE","name":"Global Medical REIT Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GMRE^A","name":"Global Medical REIT Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GMS","name":"GMS Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GMTA","name":"GATX Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GMZ","name":"Goldman Sachs MLP Income Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GNC","name":"GNC Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GNE","name":"Genie Energy Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GNE^A","name":"Genie Energy Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GNK","name":"Genco Shipping & Trading Limited ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GNL","name":"Global Net Lease, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GNL^A","name":"Global Net Lease, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GNRC","name":"Generac Holdlings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GNT","name":"GAMCO Natural Resources, Gold & Income Tust ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GNT^A","name":"GAMCO Natural Resources, Gold & Income Tust ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GNW","name":"Genworth Financial Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GOF","name":"Guggenheim Strategic Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GOL","name":"Gol Linhas Aereas Inteligentes S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GOLD","name":"Barrick Gold Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GOLF","name":"Acushnet Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GOOS","name":"Canada Goose Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GPC","name":"Genuine Parts Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GPI","name":"Group 1 Automotive, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GPJA","name":"Georgia Power Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GPK","name":"Graphic Packaging Holding Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GPM","name":"Guggenheim Enhanced Equity Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GPMT","name":"Granite Point Mortgage Trust Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GPN","name":"Global Payments Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GPRK","name":"Geopark Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GPS","name":"Gap, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GPX","name":"GP Strategies Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GRA","name":"W.R. Grace & Co.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GRAF","name":"Graf Industrial Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GRAF.U","name":"Graf Industrial Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GRAF.WS","name":"Graf Industrial Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GRAM","name":"Grana y Montero S.A.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GRC","name":"Gorman-Rupp Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GRP.U","name":"Granite Real Estate Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GRUB","name":"GrubHub Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GRX","name":"The Gabelli Healthcare & Wellness Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GRX^A","name":"The Gabelli Healthcare & Wellness Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GRX^B","name":"The Gabelli Healthcare & Wellness Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GS","name":"Goldman Sachs Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GS^A","name":"Goldman Sachs Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GSAH","name":"GS Acquisition Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GSAH.U","name":"GS Acquisition Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GSAH.WS","name":"GS Acquisition Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GSBD","name":"Goldman Sachs BDC, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GS^C","name":"Goldman Sachs Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GS^D","name":"Goldman Sachs Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GSH","name":"Guangshen Railway Company Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GS^J","name":"Goldman Sachs Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GSK","name":"GlaxoSmithKline PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GS^K","name":"Goldman Sachs Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GSL","name":"Global Ship Lease, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GSL^B","name":"Global Ship Lease, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GS^N","name":"Goldman Sachs Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GSX","name":"GSX Techedu Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GTES","name":"Gates Industrial Corporation plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GTN","name":"Gray Television, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GTN.A","name":"Gray Television, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GTS","name":"Triple-S Management Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GTT","name":"GTT Communications, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GTX","name":"Garrett Motion Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GTY","name":"Getty Realty Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GUT","name":"Gabelli Utility Trust (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GUT^A","name":"Gabelli Utility Trust (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GUT^C","name":"Gabelli Utility Trust (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GVA","name":"Granite Construction Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GWB","name":"Great Western Bancorp, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GWR","name":"Genesee & Wyoming, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GWRE","name":"Guidewire Software, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GWW","name":"W.W. Grainger, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GYB","name":"CABCO Series 2004-101 Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"GYC","name":"Corporate Asset Backed Corp CABCO","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"H","name":"Hyatt Hotels Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HAE","name":"Haemonetics Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HAL","name":"Halliburton Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HASI","name":"Hannon Armstrong Sustainable Infrastructure Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HBB","name":"Hamilton Beach Brands Holding Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HBI","name":"Hanesbrands Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HBM","name":"Hudbay Minerals Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HCA","name":"HCA Healthcare, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HCC","name":"Warrior Met Coal, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HCFT","name":"Hunt Companies Finance Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HCHC","name":"HC2 Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HCI","name":"HCI Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HCP","name":"HCP, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HCR","name":"Hi-Crush Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HCXY","name":"Hercules Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HCXZ","name":"Hercules Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HD","name":"Home Depot, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HDB","name":"HDFC Bank Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HE","name":"Hawaiian Electric Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HEI","name":"Heico Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HEI.A","name":"Heico Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HEP","name":"Holly Energy Partners, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HEQ","name":"John Hancock Hedged Equity & Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HES","name":"Hess Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HESM","name":"Hess Midstream Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HEXO","name":"HEXO Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HFC","name":"HollyFrontier Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HFRO","name":"Highland Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HFRO^A","name":"Highland Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HGH","name":"Hartford Financial Services Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HGLB","name":"Highland Global Allocation Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HGV","name":"Hilton Grand Vacations Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HHC","name":"Howard Hughes Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HHS","name":"Harte-Hanks, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HI","name":"Hillenbrand Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HIE","name":"Miller/Howard High Income Equity Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HIG","name":"Hartford Financial Services Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HIG^G","name":"Hartford Financial Services Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HII","name":"Huntington Ingalls Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HIL","name":"Hill International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HIO","name":"Western Asset High Income Opportunity Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HIW","name":"Highwoods Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HIX","name":"Western Asset High Income Fund II Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HJV","name":"MS Structured Asset Corp Saturns GE Cap Corp Series 2002-14","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HKIB","name":"AMTD International Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HL","name":"Hecla Mining Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HL^B","name":"Hecla Mining Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HLF","name":"Herbalife Nutrition Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HLI","name":"Houlihan Lokey, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HLT","name":"Hilton Worldwide Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HLX","name":"Helix Energy Solutions Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HMC","name":"Honda Motor Company, Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HMI","name":"Huami Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HMLP","name":"Hoegh LNG Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HMLP^A","name":"Hoegh LNG Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HMN","name":"Horace Mann Educators Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HMY","name":"Harmony Gold Mining Company Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HNGR","name":"Hanger, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HNI","name":"HNI Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HNP","name":"Huaneng Power International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HOG","name":"Harley-Davidson, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HOME","name":"At Home Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HON","name":"Honeywell International Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HOS","name":"Hornbeck Offshore Services","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HOV","name":"Hovnanian Enterprises Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HP","name":"Helmerich & Payne, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HPE","name":"Hewlett Packard Enterprise Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HPF","name":"John Hancock Pfd Income Fund II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HPI","name":"John Hancock Preferred Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HPP","name":"Hudson Pacific Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HPQ","name":"HP Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HPR","name":"HighPoint Resources Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HPS","name":"John Hancock Preferred Income Fund III","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HQH","name":"Tekla Healthcare Investors","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HQL","name":"Tekla Life Sciences Investors","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HR","name":"Healthcare Realty Trust Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HRB","name":"H&R Block, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HRC","name":"Hill-Rom Holdings Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HRI","name":"Herc Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HRL","name":"Hormel Foods Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HRTG","name":"Heritage Insurance Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HSBC","name":"HSBC Holdings plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HSBC^A","name":"HSBC Holdings plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HSC","name":"Harsco Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HST","name":"Host Hotels & Resorts, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HSY","name":"Hershey Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HT","name":"Hersha Hospitality Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HTA","name":"Healthcare Trust of America, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HT^C","name":"Hersha Hospitality Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HTD","name":"John Hancock Tax Advantaged Dividend Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HT^D","name":"Hersha Hospitality Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HT^E","name":"Hersha Hospitality Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HTFA","name":"Horizon Technology Finance Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HTGC","name":"Hercules Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HTH","name":"Hilltop Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HTY","name":"John Hancock Tax-Advantaged Global Shareholder Yield Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HTZ","name":"Hertz Global Holdings, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HUBB","name":"Hubbell Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HUBS","name":"HubSpot, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HUD","name":"Hudson Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HUM","name":"Humana Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HUN","name":"Huntsman Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HUYA","name":"HUYA Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HVT","name":"Haverty Furniture Companies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HVT.A","name":"Haverty Furniture Companies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HXL","name":"Hexcel Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HY","name":"Hyster-Yale Materials Handling, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HYB","name":"New America High Income Fund, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HYI","name":"Western Asset High Yield Defined Opportunity Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HYT","name":"Blackrock Corporate High Yield Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HZN","name":"Horizon Global Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"HZO","name":"MarineMax, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"I","name":"Intelsat S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IAA","name":"IAA, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IAE","name":"Voya Asia Pacific High Dividend Equity Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IAG","name":"Iamgold Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IBA","name":"Industrias Bachoco, S.A. de C.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IBM","name":"International Business Machines Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IBN","name":"ICICI Bank Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IBP","name":"Installed Building Products, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ICD","name":"Independence Contract Drilling, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ICE","name":"Intercontinental Exchange Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ICL","name":"Israel Chemicals Shs","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IDA","name":"IDACORP, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IDE","name":"Voya Infrastructure, Industrials and Materials Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IDT","name":"IDT Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IEX","name":"IDEX Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IFF","name":"International Flavors & Fragrances, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IFFT","name":"International Flavors & Fragrances, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IFN","name":"India Fund, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IFS","name":"Intercorp Financial Services Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IGA","name":"Voya Global Advantage and Premium Opportunity Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IGD","name":"Voya Global Equity Dividend and Premium Opportunity Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IGI","name":"Western Asset Investment Grade Defined Opportunity Trust Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IGR","name":"CBRE Clarion Global Real Estate Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IGT","name":"International Game Technology","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IHC","name":"Independence Holding Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IHD","name":"Voya Emerging Markets High Income Dividend Equity Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IHG","name":"Intercontinental Hotels Group","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IHIT","name":"Invesco High Income 2023 Target Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IHTA","name":"Invesco High Income 2024 Target Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IID","name":"Voya International High Dividend Equity Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IIF","name":"Morgan Stanley India Investment Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IIM","name":"Invesco Value Municipal Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IIPR","name":"Innovative Industrial Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IIPR^A","name":"Innovative Industrial Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IMAX","name":"Imax Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INB","name":"Cohen & Steers Global Income Builder, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INF","name":"Brookfield Global Listed Infrastructure Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INFO","name":"IHS Markit Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INFY","name":"Infosys Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ING","name":"ING Group, N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INGR","name":"Ingredion Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INN","name":"Summit Hotel Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INN^D","name":"Summit Hotel Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INN^E","name":"Summit Hotel Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INSI","name":"Insight Select Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INSP","name":"Inspire Medical Systems, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INST","name":"Instructure, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INSW","name":"International Seaways, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INSW^A","name":"International Seaways, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INT","name":"World Fuel Services Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INVH","name":"Invitation Homes Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"INXN","name":"InterXion Holding N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IO","name":"Ion Geophysical Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IP","name":"International Paper Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IPG","name":"Interpublic Group of Companies, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IPHI","name":"Inphi Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IPI","name":"Intrepid Potash, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IPOA","name":"Social Capital Hedosophia Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IPOA.U","name":"Social Capital Hedosophia Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IPOA.WS","name":"Social Capital Hedosophia Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IQI","name":"Invesco Quality Municipal Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IQV","name":"IQVIA Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IR","name":"Ingersoll-Rand plc (Ireland)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IRET","name":"Investors Real Estate Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IRET^C","name":"Investors Real Estate Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IRL","name":"New Ireland Fund, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IRM","name":"Iron Mountain Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IRR","name":"Voya Natural Resources Equity Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IRS","name":"IRSA Inversiones Y Representaciones S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IRT","name":"Independence Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ISD","name":"PGIM High Yield Bond Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ISG","name":"ING Group, N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IT","name":"Gartner, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ITCB","name":"Ita? CorpBanca","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ITGR","name":"Integer Holdings Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ITT","name":"ITT Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ITUB","name":"Itau Unibanco Banco Holding SA","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ITW","name":"Illinois Tool Works Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IVC","name":"Invacare Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IVH","name":"Ivy High Income Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IVR","name":"INVESCO MORTGAGE CAPITAL INC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IVR^A","name":"Invesco Mortgage Capital Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IVR^B","name":"INVESCO MORTGAGE CAPITAL INC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IVR^C","name":"INVESCO MORTGAGE CAPITAL INC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IVZ","name":"Invesco Plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"IX","name":"Orix Corp Ads","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JAG","name":"Jagged Peak Energy Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JAX","name":"J. Alexander&#39;s Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JBGS","name":"JBG SMITH Properties","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JBK","name":"Lehman ABS Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JBL","name":"Jabil Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JBN","name":"Select Asset Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JBR","name":"Select Asset Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JBT","name":"John Bean Technologies Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JCAP","name":"Jernigan Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JCAP^B","name":"Jernigan Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JCE","name":"Nuveen Core Equity Alpha Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JCI","name":"Johnson Controls International plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JCO","name":"Nuveen Credit Opportunities 2022 Target Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JCP","name":"J.C. Penney Company, Inc. Holding Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JDD","name":"Nuveen Diversified Dividend and Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JE","name":"Just Energy Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JE^A","name":"Just Energy Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JEC","name":"Jacobs Engineering Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JEF","name":"Jefferies Financial Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JELD","name":"JELD-WEN Holding, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JEMD","name":"Nuveen Emerging Markets Debt 2022 Target Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JEQ","name":"Aberdeen Japan Equity Fund, Inc. ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JFR","name":"Nuveen Floating Rate Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JGH","name":"Nuveen Global High Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JHAA","name":"Nuveen High Income 2023 Target Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JHB","name":"Nuveen High Income November 2021 Target Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JHD","name":"Nuveen High Income December 2019 Target Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JHG","name":"Janus Henderson Group plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JHI","name":"John Hancock Investors Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JHS","name":"John Hancock Income Securities Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JHX","name":"James Hardie Industries plc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JHY","name":"Nuveen High Income 2020 Target Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JILL","name":"J. Jill, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JKS","name":"JinkoSolar Holding Company Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JLL","name":"Jones Lang LaSalle Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JLS","name":"Nuveen Mortgage Opportunity Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JMEI","name":"Jumei International Holding Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JMF","name":"Nuveen Energy MLP Total Return Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JMIA","name":"Jumia Technologies AG","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JMLP","name":"Nuveen All Cap Energy MLP Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JMM","name":"Nuveen Multi-Market Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JMP","name":"JMP Group LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JMPB","name":"JMP Group LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JMPD","name":"JMP Group LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JMT","name":"Nuven Mortgage Opportunity Term Fund 2","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JNJ","name":"Johnson & Johnson","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JNPR","name":"Juniper Networks, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JOE","name":"St. Joe Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JOF","name":"Japan Smaller Capitalization Fund Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JP","name":"Jupai Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JPC","name":"Nuveen Preferred & Income Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JPI","name":"Nuveen Preferred and Income Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JPM","name":"J P Morgan Chase & Co","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JPM^A","name":"J P Morgan Chase & Co","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JPM^C","name":"J P Morgan Chase & Co","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JPM^D","name":"J P Morgan Chase & Co","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JPM^F","name":"J P Morgan Chase & Co","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JPM^G","name":"J P Morgan Chase & Co","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JPM^H","name":"J P Morgan Chase & Co","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JPS","name":"Nuveen Preferred & Income Securities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JPT","name":"Nuveen Preferred and Income 2022 Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JQC","name":"Nuveen Credit Strategies Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JRI","name":"Nuveen Real Asset Income and Growth Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JRO","name":"Nuveen Floating Rate Income Opportuntiy Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JRS","name":"Nuveen Real Estate Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JSD","name":"Nuveen Short Duration Credit Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JT","name":"Jianpu Technology Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JTA","name":"Nuveen Tax-Advantaged Total Return Strategy Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JTD","name":"Nuveen Tax-Advantaged Dividend Growth Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JW.A","name":"John Wiley & Sons, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JW.B","name":"John Wiley & Sons, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"JWN","name":"Nordstrom, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"K","name":"Kellogg Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KAI","name":"Kadant Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KAMN","name":"Kaman Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KAR","name":"KAR Auction Services, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KB","name":"KB Financial Group Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KBH","name":"KB Home","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KBR","name":"KBR, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KDMN","name":"Kadmon Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KDP","name":"Keurig Dr Pepper Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KEG","name":"Key Energy Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KEM","name":"Kemet Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KEN","name":"Kenon Holdings Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KEP","name":"Korea Electric Power Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KEX","name":"Kirby Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KEY","name":"KeyCorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KEY^I","name":"KeyCorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KEY^J","name":"KeyCorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KEY^K","name":"KeyCorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KEYS","name":"Keysight Technologies Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KF","name":"Korea Fund, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KFS","name":"Kingsway Financial Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KFY","name":"Korn Ferry ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KGC","name":"Kinross Gold Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KIM","name":"Kimco Realty Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KIM^I.CL","name":"Kimco Realty Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KIM^J","name":"Kimco Realty Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KIM^K.CL","name":"Kimco Realty Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KIM^L","name":"Kimco Realty Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KIM^M","name":"Kimco Realty Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KIO","name":"KKR Income Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KKR","name":"KKR & Co. Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KKR^A","name":"KKR & Co. Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KKR^B","name":"KKR & Co. Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KL","name":"Kirkland Lake Gold Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KMB","name":"Kimberly-Clark Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KMF","name":"Kayne Anderson Midstream Energy Fund, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KMI","name":"Kinder Morgan, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KMPR","name":"Kemper Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KMT","name":"Kennametal Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KMX","name":"CarMax Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KN","name":"Knowles Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KNL","name":"Knoll, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KNOP","name":"KNOT Offshore Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KNX","name":"Knight Transportation, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KO","name":"Coca-Cola Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KODK","name":"Eastman Kodak Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KOF","name":"Coca Cola Femsa S.A.B. de C.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KOP","name":"Koppers Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KOS","name":"Kosmos Energy Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KR","name":"Kroger Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KRA","name":"Kraton Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KRC","name":"Kilroy Realty Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KREF","name":"KKR Real Estate Finance Trust Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KRG","name":"Kite Realty Group Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KRO","name":"Kronos Worldwide Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KRP","name":"Kimbell Royalty Partners","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KSM","name":"Scudder Strategic Municiple Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KSS","name":"Kohl&#39;s Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KSU","name":"Kansas City Southern","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KSU^","name":"Kansas City Southern","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KT","name":"KT Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KTB","name":"Kontoor Brands, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KTF","name":"Scudder Municiple Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KTH","name":"Lehman ABS Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KTN","name":"Lehman ABS Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KTP","name":"Lehman ABS Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KW","name":"Kennedy-Wilson Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KWR","name":"Quaker Chemical Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KYN","name":"Kayne Anderson MLP/Midstream Investment Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"KYN^F","name":"Kayne Anderson MLP/Midstream Investment Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"L","name":"Loews Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LAC","name":"Lithium Americas Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LAD","name":"Lithia Motors, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LADR","name":"Ladder Capital Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LAIX","name":"LAIX Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LAZ","name":"Lazard Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LB","name":"L Brands, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LBRT","name":"Liberty Oilfield Services Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LC","name":"LendingClub Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LCI","name":"Lannett Co Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LCII","name":"LCI Industries ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LDL","name":"Lydall, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LDOS","name":"Leidos Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LDP","name":"Cohen & Steers Limited Duration Preferred and Income Fund, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LEA","name":"Lear Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LEAF","name":"Leaf Group Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LEE","name":"Lee Enterprises, Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LEG","name":"Leggett & Platt, Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LEJU","name":"Leju Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LEN","name":"Lennar Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LEN.B","name":"Lennar Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LEO","name":"BNY Mellon Strategic Municipals, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LEVI","name":"Levi Strauss & Co","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LFC","name":"China Life Insurance Company Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LGC","name":"Legacy Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LGC.U","name":"Legacy Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LGC.WS","name":"Legacy Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LGF.A","name":"Lions Gate Entertainment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LGF.B","name":"Lions Gate Entertainment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LGI","name":"Lazard Global Total Return and Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LH","name":"Laboratory Corporation of America Holdings","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LHC","name":"Leo Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LHC.U","name":"Leo Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LHC.WS","name":"Leo Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LHX","name":"L3Harris Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LII","name":"Lennox International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LIN","name":"Linde plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LINX","name":"Linx S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LITB","name":"LightInTheBox Holding Co., Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LKSD","name":"LSC Communications, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LL","name":"Lumber Liquidators Holdings, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LLY","name":"Eli Lilly and Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LM","name":"Legg Mason, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LMHA","name":"Legg Mason, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LMHB","name":"Legg Mason, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LMT","name":"Lockheed Martin Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LN","name":"LINE Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LNC","name":"Lincoln National Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LND","name":"Brasilagro Cia Brasileira De Propriedades Agricolas","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LNN","name":"Lindsay Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LOMA","name":"Loma Negra Compania Industrial Argentina Sociedad Anonima","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LOR","name":"Lazard World Dividend & Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LOW","name":"Lowe&#39;s Companies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LPG","name":"Dorian LPG Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LPI","name":"Laredo Petroleum, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LPL","name":"LG Display Co., Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LPT","name":"Liberty Property Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LPX","name":"Louisiana-Pacific Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LRN","name":"K12 Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LSI","name":"Life Storage, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LTC","name":"LTC Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LTHM","name":"Livent Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LTM","name":"LATAM Airlines Group S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LUB","name":"Luby&#39;s, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LUV","name":"Southwest Airlines Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LVS","name":"Las Vegas Sands Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LW","name":"Lamb Weston Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LXFR","name":"Luxfer Holdings PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LXP","name":"Lexington Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LXP^C","name":"Lexington Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LXU","name":"Lsb Industries Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LYB","name":"LyondellBasell Industries NV","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LYG","name":"Lloyds Banking Group Plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LYV","name":"Live Nation Entertainment, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"LZB","name":"La-Z-Boy Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"M","name":"Macy&#39;s Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MA","name":"Mastercard Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MAA","name":"Mid-America Apartment Communities, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MAA^I","name":"Mid-America Apartment Communities, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MAC","name":"Macerich Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MAIN","name":"Main Street Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MAN","name":"ManpowerGroup","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MANU","name":"Manchester United Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MAS","name":"Masco Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MATX","name":"Matson, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MAV","name":"Pioneer Municipal High Income Advantage Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MAXR","name":"Maxar Technologies Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MBI","name":"MBIA, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MBT","name":"Mobile TeleSystems OJSC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MC","name":"Moelis & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCA","name":"Blackrock MuniYield California Insured Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCB","name":"Metropolitan Bank Holding Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCC","name":"Medley Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCD","name":"McDonald&#39;s Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCI","name":"Barings Corporate Investors","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCK","name":"McKesson Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCN","name":"Madison Covered Call & Equity Strategy Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCO","name":"Moody&#39;s Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCR","name":"MFS Charter Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCRN","name":"Milacron Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCS","name":"Marcus Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCV","name":"Medley Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCX","name":"Medley Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MCY","name":"Mercury General Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MD","name":"Mednax, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MDC","name":"M.D.C. Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MDLA","name":"Medallia, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MDLQ","name":"Medley LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MDLX","name":"Medley LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MDLY","name":"Medley Management Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MDP","name":"Meredith Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MDR","name":"McDermott International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MDT","name":"Medtronic plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MDU","name":"MDU Resources Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MEC","name":"Mayville Engineering Company, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MED","name":"MEDIFAST INC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MEI","name":"Methode Electronics, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MEN","name":"Blackrock MuniEnhanced Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MER^K","name":"Merrill Lynch & Co., Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MET","name":"MetLife, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MET^A","name":"MetLife, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MET^E","name":"MetLife, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFA","name":"MFA Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFA^B","name":"MFA Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFAC","name":"Megalith Financial Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFAC.U","name":"Megalith Financial Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFAC.WS","name":"Megalith Financial Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFC","name":"Manulife Financial Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFD","name":"Macquarie First Trust Global","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFG","name":"Mizuho Financial Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFGP","name":"Micro Focus Intl PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFL","name":"Blackrock MuniHoldings Investment Quality Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFM","name":"MFS Municipal Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFO","name":"MFA Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFT","name":"Blackrock MuniYield Investment QualityFund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MFV","name":"MFS Special Value Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MG","name":"Mistras Group Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MGA","name":"Magna International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MGF","name":"MFS Government Markets Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MGM","name":"MGM Resorts International","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MGP","name":"MGM Growth Properties LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MGR","name":"Affiliated Managers Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MGU","name":"Macquarie Global Infrastructure Total Return Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MGY","name":"Magnolia Oil & Gas Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MH^A","name":"Maiden Holdings, Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MH^C","name":"Maiden Holdings, Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MHD","name":"Blackrock MuniHoldings Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MH^D","name":"Maiden Holdings, Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MHE","name":"BlackRock Massachusetts Tax-Exempt Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MHF","name":"Western Asset Municipal High Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MHI","name":"Pioneer Municipal High Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MHK","name":"Mohawk Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MHLA","name":"Maiden Holdings, Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MHN","name":"Blackrock MuniHoldings New York Quality Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MHNC","name":"Maiden Holdings, Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MHO","name":"M/I Homes, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MIC","name":"Macquarie Infrastructure Corporation ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MIE","name":"Cohen & Steers MLP Income and Energy Opportunity Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MIN","name":"MFS Intermediate Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MITT","name":"AG Mortgage Investment Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MITT^A","name":"AG Mortgage Investment Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MITT^B","name":"AG Mortgage Investment Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MIXT","name":"MiX Telematics Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MIY","name":"Blackrock MuniYield Michigan Quality Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MKC","name":"McCormick & Company, Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MKC.V","name":"McCormick & Company, Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MKL","name":"Markel Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MLI","name":"Mueller Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MLM","name":"Martin Marietta Materials, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MLP","name":"Maui Land & Pineapple Company, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MLR","name":"Miller Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MMC","name":"Marsh & McLennan Companies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MMD","name":"MainStay MacKay DefinedTerm Municipal Opportunitie","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MMI","name":"Marcus & Millichap, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MMM","name":"3M Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MMP","name":"Magellan Midstream Partners L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MMS","name":"Maximus, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MMT","name":"MFS Multimarket Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MMU","name":"Western Asset Managed Municipals Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MN","name":"Manning & Napier, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MNE","name":"Blackrock Muni New York Intermediate Duration Fund Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MNK","name":"Mallinckrodt plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MNP","name":"Western Asset Municipal Partners Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MNR","name":"Monmouth Real Estate Investment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MNR^C","name":"Monmouth Real Estate Investment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MNRL","name":"Brigham Minerals, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MO","name":"Altria Group","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MOD","name":"Modine Manufacturing Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MODN","name":"Model N, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MOG.A","name":"Moog Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MOG.B","name":"Moog Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MOGU","name":"MOGU Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MOH","name":"Molina Healthcare Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MOS","name":"Mosaic Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MOSC","name":"Mosaic Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MOSC.U","name":"Mosaic Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MOSC.WS","name":"Mosaic Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MOV","name":"Movado Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MPA","name":"Blackrock MuniYield Pennsylvania Quality Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MPC","name":"Marathon Petroleum Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MPLX","name":"MPLX LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MPV","name":"Barings Participation Investors","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MPW","name":"Medical Properties Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MPX","name":"Marine Products Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MQT","name":"Blackrock MuniYield Quality Fund II, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MQY","name":"Blackrock MuniYield Quality Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MR","name":"Montage Resources Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MRC","name":"MRC Global Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MRK","name":"Merck & Company, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MRO","name":"Marathon Oil Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MS","name":"Morgan Stanley","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MSA","name":"MSA Safety Incorporporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MS^A","name":"Morgan Stanley","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MSB","name":"Mesabi Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MSC","name":"Studio City International Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MSCI","name":"MSCI Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MSD","name":"Morgan Stanley Emerging Markets Debt Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MS^E","name":"Morgan Stanley","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MS^F","name":"Morgan Stanley","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MSG","name":"The Madison Square Garden Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MS^G","name":"Morgan Stanley","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MSGN","name":"MSG Networks Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MSI","name":"Motorola Solutions, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MS^I","name":"Morgan Stanley","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MS^K","name":"Morgan Stanley","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MSL","name":"MidSouth Bancorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MSM","name":"MSC Industrial Direct Company, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MT","name":"ArcelorMittal","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTB","name":"M&T Bank Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTD","name":"Mettler-Toledo International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTDR","name":"Matador Resources Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTG","name":"MGIC Investment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTH","name":"Meritage Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTL","name":"Mechel PAO","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTL^","name":"Mechel PAO","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTN","name":"Vail Resorts, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTOR","name":"Meritor, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTR","name":"Mesa Royalty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTRN","name":"Materion Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTT","name":"Western Asset Municipal Defined Opportunity Trust Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTW","name":"Manitowoc Company, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTX","name":"Minerals Technologies Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MTZ","name":"MasTec, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MUA","name":"Blackrock MuniAssets Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MUC","name":"Blackrock MuniHoldings California Quality Fund,  Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MUE","name":"Blackrock MuniHoldings Quality Fund II, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MUFG","name":"Mitsubishi UFJ Financial Group Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MUH","name":"Blackrock MuniHoldings Fund II, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MUI","name":"Blackrock Muni Intermediate Duration Fund Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MUJ","name":"Blackrock MuniHoldings New Jersey Insured Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MUR","name":"Murphy Oil Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MUS","name":"Blackrock MuniHoldings Quality Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MUSA","name":"Murphy USA Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MUX","name":"McEwen Mining Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MVC","name":"MVC Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MVCD","name":"MVC Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MVF","name":"MuniVest Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MVO","name":"MV Oil Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MVT","name":"Blackrock MuniVest Fund II, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MWA","name":"Mueller Water Products Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MX","name":"MagnaChip Semiconductor Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MXE","name":"Mexico Equity and Income Fund, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MXF","name":"Mexico Fund, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MXL","name":"MaxLinear, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MYC","name":"Blackrock MuniYield California Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MYD","name":"Blackrock MuniYield Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MYE","name":"Myers Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MYF","name":"Blackrock MuniYield Investment Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MYI","name":"Blackrock MuniYield Quality Fund III, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MYJ","name":"Blackrock MuniYield New Jersey Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MYN","name":"Blackrock MuniYield New York Quality Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MYOV","name":"Myovant Sciences Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"MZA","name":"MuniYield Arizona Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NAC","name":"Nuveen California Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NAD","name":"Nuveen Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NAN","name":"Nuveen New York Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NAT","name":"Nordic American Tankers Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NAV","name":"Navistar International Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NAV^D","name":"Navistar International Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NAZ","name":"Nuveen Arizona Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NBB","name":"Nuveen Taxable Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NBHC","name":"National Bank Holdings Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NBL","name":"Noble Energy Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NBLX","name":"Noble Midstream Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NBR","name":"Nabors Industries Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NBR^A","name":"Nabors Industries Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NC","name":"NACCO Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NCA","name":"Nuveen California Municipal Value Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NCB","name":"Nuveen California Municipal Value Fund 2","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NCI","name":"Navigant Consulting, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NCLH","name":"Norwegian Cruise Line Holdings Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NCR","name":"NCR Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NCV","name":"AllianzGI Convertible & Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NCV^A","name":"AllianzGI Convertible & Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NCZ","name":"AllianzGI Convertible & Income Fund II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NCZ^A","name":"AllianzGI Convertible & Income Fund II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NDP","name":"Tortoise Energy Independence Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NE","name":"Noble Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEA","name":"Nuveen AMT-Free Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEE","name":"NextEra Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEE^I","name":"NextEra Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEE^J","name":"NextEra Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEE^K","name":"NextEra Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEE^N","name":"NextEra Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEE^O","name":"NextEra Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEM","name":"Newmont Goldcorp Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEP","name":"NextEra Energy Partners, LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEU","name":"NewMarket Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEV","name":"Nuveen Enhanced Municipal Value Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEW","name":"Puxin Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEWM","name":"New Media Investment Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEWR","name":"New Relic, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NEXA","name":"Nexa Resources S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NFC","name":"New Frontier Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NFC.U","name":"New Frontier Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NFC.WS","name":"New Frontier Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NFG","name":"National Fuel Gas Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NFJ","name":"AllianzGI NFJ Dividend, Interest & Premium Strategy Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NGG","name":"National Grid Transco, PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NGL","name":"NGL ENERGY PARTNERS LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NGL^B","name":"NGL ENERGY PARTNERS LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NGL^C","name":"NGL ENERGY PARTNERS LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NGLS^A","name":"Targa Resources Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NGS","name":"Natural Gas Services Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NGVC","name":"Natural Grocers by Vitamin Cottage, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NGVT","name":"Ingevity Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NHA","name":"Nuveen Municipal 2021 Target Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NHF","name":"NexPoint Strategic Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NHI","name":"National Health Investors, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NI","name":"NiSource, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NI^B","name":"NiSource, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NID","name":"Nuveen Intermediate Duration Municipal Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NIE","name":"AllianzGI Equity & Convertible Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NIM","name":"Nuveen Select Maturities Municipal Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NINE","name":"Nine Energy Service, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NIO","name":"NIO Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NIQ","name":"Nuveenn Intermediate Duration Quality Municipal Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NJR","name":"NewJersey Resources Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NJV","name":"Nuveen New Jersey Municipal Value Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NKE","name":"Nike, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NKG","name":"Nuveen Georgia Quality Municipal Income Fund ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NKX","name":"Nuveen California AMT-Free Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NL","name":"NL Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NLS","name":"Nautilus Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NLSN","name":"Nielsen N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NLY","name":"Annaly Capital Management Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NLY^D","name":"Annaly Capital Management Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NLY^F","name":"Annaly Capital Management Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NLY^G","name":"Annaly Capital Management Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NLY^I","name":"Annaly Capital Management Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NM","name":"Navios Maritime Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NMFC","name":"New Mountain Finance Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NMFX","name":"New Mountain Finance Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NM^G","name":"Navios Maritime Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NM^H","name":"Navios Maritime Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NMI","name":"Nuveen Municipal Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NMK^B","name":"Niagara Mohawk Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NMK^C","name":"Niagara Mohawk Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NMM","name":"Navios Maritime Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NMR","name":"Nomura Holdings Inc ADR","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NMS","name":"Nuveen Minnesota Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NMT","name":"Nuveen Massachusetts Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NMY","name":"Nuveen Maryland Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NMZ","name":"Nuveen Municipal High Income Opportunity Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NNA","name":"Navios Maritime Acquisition Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NNC","name":"Nuveen North Carolina Quality Municipal Income Fd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NNI","name":"Nelnet, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NNN","name":"National Retail Properties","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NNN^E.CL","name":"National Retail Properties","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NNN^F","name":"National Retail Properties","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NNY","name":"Nuveen New York Municipal Value Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NOA","name":"North American Construction Group Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NOAH","name":"Noah Holdings Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NOC","name":"Northrop Grumman Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NOK","name":"Nokia Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NOM","name":"Nuveen Missouri Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NOMD","name":"Nomad Foods Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NOV","name":"National Oilwell Varco, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NOVA","name":"Sunnova Energy International Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NOW","name":"ServiceNow, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NP","name":"Neenah, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NPK","name":"National Presto Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NPN","name":"Nuveen Pennsylvania Municipal Value Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NPO","name":"EnPro Industries","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NPTN","name":"NeoPhotonics Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NPV","name":"Nuveen Virginia Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NQP","name":"Nuveen Pennsylvania Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NR","name":"Newpark Resources, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NRE","name":"NorthStar Realty Europe Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NRG","name":"NRG Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NRGX","name":"PIMCO Energy and Tactical Credit Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NRK","name":"Nuveen New York AMT-Free Quality Municipal","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NRP","name":"Natural Resource Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NRT","name":"North European Oil Royality Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NRUC","name":"National Rural Utilities Cooperative Finance Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NRZ","name":"New Residential Investment Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NRZ^A","name":"New Residential Investment Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NRZ^B","name":"New Residential Investment Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NS","name":"Nustar Energy L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NSA","name":"National Storage Affiliates Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NS^A","name":"Nustar Energy L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NSA^A","name":"National Storage Affiliates Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NS^B","name":"Nustar Energy L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NSC","name":"Norfolk Southern Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NS^C","name":"Nustar Energy L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NSCO","name":"Nesco Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NSCO.WS","name":"Nesco Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NSL","name":"Nuveen Senior Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NSP","name":"Insperity, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NSS","name":"NuStar Logistics, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NTB","name":"Bank of N.T. Butterfield & Son Limited (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NTC","name":"Nuveen Connecticut Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NTEST","name":"NASDAQ TEST STOCK","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NTEST.A","name":"NASDAQ TEST STOCK","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NTEST.B","name":"NASDAQ TEST STOCK","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NTEST.C","name":"NASDAQ TEST STOCK","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NTG","name":"Tortoise Midstream Energy Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NTP","name":"Nam Tai Property Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NTR","name":"Nutrien Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NTX","name":"Nuveen Texas Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NTZ","name":"Natuzzi, S.p.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NUE","name":"Nucor Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NUM","name":"Nuveen Michigan Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NUO","name":"Nuveen Ohio Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NUS","name":"Nu Skin Enterprises, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NUV","name":"Nuveen AMT-Free Municipal Value Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NUW","name":"Nuveen AMT-Free Municipal Value Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NVG","name":"Nuveen AMT-Free Municipal Credit Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NVGS","name":"Navigator Holdings Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NVO","name":"Novo Nordisk A/S","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NVR","name":"NVR, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NVRO","name":"Nevro Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NVS","name":"Novartis AG","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NVT","name":"nVent Electric plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NVTA","name":"Invitae Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NWE","name":"NorthWestern Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NWHM","name":"New Home Company Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NWN","name":"Northwest Natural Holding Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NX","name":"Quanex Building Products Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NXC","name":"Nuveen Insured California Select Tax-Free Income Portfolio","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NXJ","name":"Nuveen New Jersey Quality Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NXN","name":"Nuveen Insured New York Select Tax-Free Income Portfolio","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NXP","name":"Nuveen Select Tax Free Income Portfolio","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NXQ","name":"Nuveen Select Tax Free Income Portfolio II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NXR","name":"Nuveen Select Tax Free Income Portfolio III","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NXRT","name":"NexPoint Residential Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NYCB","name":"New York Community Bancorp, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NYCB^A","name":"New York Community Bancorp, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NYCB^U","name":"New York Community Bancorp, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NYT","name":"New York Times Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NYV","name":"Nuveen New York Municipal Value Fund 2","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"NZF","name":"Nuveen Municipal Credit Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"O","name":"Realty Income Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OAC","name":"Oaktree Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OAC.U","name":"Oaktree Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OAC.WS","name":"Oaktree Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OAK","name":"Oaktree Capital Group, LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OAK^A","name":"Oaktree Capital Group, LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OAK^B","name":"Oaktree Capital Group, LLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OAS","name":"Oasis Petroleum Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OBE","name":"Obsidian Energy Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OC","name":"Owens Corning Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OCN","name":"Ocwen Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ODC","name":"Oil-Dri Corporation Of America","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OEC","name":"Orion Engineered Carbons S.A","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OFC","name":"Corporate Office Properties Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OFG","name":"OFG Bancorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OFG^A","name":"OFG Bancorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OFG^B","name":"OFG Bancorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OFG^D","name":"OFG Bancorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OGE","name":"OGE Energy Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OGS","name":"ONE Gas, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OHI","name":"Omega Healthcare Investors, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OI","name":"Owens-Illinois, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OIA","name":"Invesco Municipal Income Opportunities Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OIBR.C","name":"Oi S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OII","name":"Oceaneering International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OIS","name":"Oil States International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OKE","name":"ONEOK, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OLN","name":"Olin Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OLP","name":"One Liberty Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OMC","name":"Omnicom Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OMF","name":"OneMain Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OMI","name":"Owens & Minor, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OMN","name":"OMNOVA Solutions Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OMP","name":"Oasis Midstream Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ONDK","name":"On Deck Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ONE","name":"OneSmart International Education Group Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OOMA","name":"Ooma, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OPP","name":"RiverNorth/DoubleLine Strategic Opportunity Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OPY","name":"Oppenheimer Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OR","name":"Osisko Gold Royalties Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ORA","name":"Ormat Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ORAN","name":"Orange","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ORC","name":"Orchid Island Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ORCC","name":"Owl Rock Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ORCL","name":"Oracle Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ORI","name":"Old Republic International Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ORN","name":"Orion Group Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OSB","name":"Norbord Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OSG","name":"Overseas Shipholding Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OSK","name":"Oshkosh Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OSLE","name":"Oaktree Specialty Lending Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OUT","name":"OUTFRONT Media Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OXM","name":"Oxford Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"OXY","name":"Occidental Petroleum Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PAA","name":"Plains All American Pipeline, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PAC","name":"Grupo Aeroportuario Del Pacifico, S.A. de C.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PACD","name":"Pacific Drilling S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PACK","name":"Ranpak Holdings Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PACK.WS","name":"Ranpak Holdings Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PAG","name":"Penske Automotive Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PAGP","name":"Plains Group Holdings, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PAGS","name":"PagSeguro Digital Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PAI","name":"Western Asset Investment Grade Income Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PAM","name":"Pampa Energia S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PANW","name":"Palo Alto Networks, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PAR","name":"PAR Technology Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PARR","name":"Par Pacific Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PAYC","name":"Paycom Software, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PB","name":"Prosperity Bancshares, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PBA","name":"Pembina Pipeline Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PBB","name":"Prospect Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PBC","name":"Prospect Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PBF","name":"PBF Energy Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PBFX","name":"PBF Logistics LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PBH","name":"Prestige Consumer Healthcare Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PBI","name":"Pitney Bowes Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PBI^B","name":"Pitney Bowes Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PBR","name":"Petroleo Brasileiro S.A.- Petrobras","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PBR.A","name":"Petroleo Brasileiro S.A.- Petrobras","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PBT","name":"Permian Basin Royalty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PBY","name":"Prospect Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PCF","name":"High Income Securities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PCG","name":"Pacific Gas & Electric Co.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PCI","name":"PIMCO Dynamic Credit and Mortgage Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PCK","name":"Pimco California Municipal Income Fund II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PCM","name":"PIMCO Commercial Mortgage Securities Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PCN","name":"Pimco Corporate & Income Stategy Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PCQ","name":"PIMCO California Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PD","name":"PagerDuty, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PDI","name":"PIMCO Dynamic Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PDM","name":"Piedmont Office Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PDS","name":"Precision Drilling Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PDT","name":"John Hancock Premium Dividend Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PE","name":"Parsley Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PEB","name":"Pebblebrook Hotel Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PEB^C","name":"Pebblebrook Hotel Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PEB^D","name":"Pebblebrook Hotel Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PEB^E","name":"Pebblebrook Hotel Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PEB^F","name":"Pebblebrook Hotel Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PEG","name":"Public Service Enterprise Group Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PEI","name":"Pennsylvania Real Estate Investment Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PEI^B","name":"Pennsylvania Real Estate Investment Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PEI^C","name":"Pennsylvania Real Estate Investment Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PEI^D","name":"Pennsylvania Real Estate Investment Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PEN","name":"Penumbra, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PEO","name":"Adams Natural Resources Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PER","name":"SandRidge Permian Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PFD","name":"Flaherty & Crumrine Preferred and Income Fund Inco","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PFE","name":"Pfizer, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PFGC","name":"Performance Food Group Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PFH","name":"CABCO Series 2004-101 Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PFL","name":"PIMCO Income Strategy Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PFN","name":"PIMCO Income Strategy Fund II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PFO","name":"Flaherty & Crumrine Preferred and Income Opportuni","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PFS","name":"Provident Financial Services, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PFSI","name":"PennyMac Financial Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PG","name":"Procter & Gamble Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PGP","name":"Pimco Global Stocksplus & Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PGR","name":"Progressive Corporation (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PGRE","name":"Paramount Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PGTI","name":"PGT Innovations, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PGZ","name":"Principal Real Estate Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PH","name":"Parker-Hannifin Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PHD","name":"Pioneer Floating Rate Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PHG","name":"Koninklijke Philips N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PHI","name":"PLDT Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PHK","name":"Pimco High Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PHM","name":"PulteGroup, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PHR","name":"Phreesia, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PHT","name":"Pioneer High Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PHX","name":"Panhandle Royalty Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PIC","name":"Pivotal Investment Corporation II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PIC.U","name":"Pivotal Investment Corporation II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PIC.WS","name":"Pivotal Investment Corporation II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PII","name":"Polaris Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PIM","name":"Putnam Master Intermediate Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PINS","name":"Pinterest, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PIR","name":"Pier 1 Imports, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PIY","name":"Merrill Lynch Depositor, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PJC","name":"Piper Jaffray Companies","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PJH","name":"Prudential Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PJT","name":"PJT Partners Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PK","name":"Park Hotels & Resorts Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PKD","name":"Parker Drilling Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PKE","name":"Park Aerospace Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PKG","name":"Packaging Corporation of America","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PKI","name":"PerkinElmer, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PKO","name":"Pimco Income Opportunity Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PKX","name":"POSCO","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PLAN","name":"Anaplan, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PLD","name":"Prologis, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PLNT","name":"Planet Fitness, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PLOW","name":"Douglas Dynamics, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PLT","name":"Plantronics, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PM","name":"Philip Morris International Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PMF","name":"PIMCO Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PML","name":"Pimco Municipal Income Fund II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PMM","name":"Putnam Managed Municipal Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PMO","name":"Putnam Municipal Opportunities Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PMT","name":"PennyMac Mortgage Investment Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PMT^A","name":"PennyMac Mortgage Investment Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PMT^B","name":"PennyMac Mortgage Investment Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PMX","name":"PIMCO Municipal Income Fund III","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PNC","name":"PNC Financial Services Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PNC^P","name":"PNC Financial Services Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PNC^Q","name":"PNC Financial Services Group, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PNF","name":"PIMCO New York Municipal Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PNI","name":"Pimco New York Municipal Income Fund II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PNM","name":"PNM Resources, Inc. (Holding Co.)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PNR","name":"Pentair plc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PNW","name":"Pinnacle West Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"POL","name":"PolyOne Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"POR","name":"Portland General Electric Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"POST","name":"Post Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PPDF","name":"PPDAI Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PPG","name":"PPG Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PPL","name":"PPL Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PPR","name":"Voya Prime Rate Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PPT","name":"Putnam Premier Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PPX","name":"PPL Capital Funding, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PQG","name":"PQ Group Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRA","name":"ProAssurance Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRE^F","name":"PartnerRe Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRE^G","name":"PartnerRe Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRE^H","name":"PartnerRe Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRE^I","name":"PartnerRe Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRGO","name":"Perrigo Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRH","name":"Prudential Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRI","name":"Primerica, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRIF^A","name":"Priority Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRIF^B","name":"Priority Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRIF^C","name":"Priority Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRIF^D","name":"Priority Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRLB","name":"Proto Labs, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRO","name":"PROS Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PROS","name":"ProSight Global, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRS","name":"Prudential Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRSP","name":"Perspecta Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRT","name":"PermRock Royalty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRTY","name":"Party City Holdco Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PRU","name":"Prudential Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSA","name":"Public Storage","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSA^A","name":"Public Storage","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSA^B","name":"Public Storage","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSA^C","name":"Public Storage","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSA^D","name":"Public Storage","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSA^E","name":"Public Storage","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSA^F","name":"Public Storage","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSA^G","name":"Public Storage","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSA^H","name":"Public Storage","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSA^V","name":"Public Storage","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSA^W","name":"Public Storage","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSA^X","name":"Public Storage","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSB","name":"PS Business Parks, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSB^U","name":"PS Business Parks, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSB^V","name":"PS Business Parks, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSB^W","name":"PS Business Parks, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSB^X","name":"PS Business Parks, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSB^Y","name":"PS Business Parks, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSF","name":"Cohen & Steers Select Preferred and Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSN","name":"Parsons Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSO","name":"Pearson, Plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSTG","name":"Pure Storage, Inc. ","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSTL","name":"Postal Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSV","name":"Hermitage Offshore Services Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSX","name":"Phillips 66","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PSXP","name":"Phillips 66 Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PTR","name":"PetroChina Company Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PTY","name":"Pimco Corporate & Income Opportunity Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PUK","name":"Prudential Public Limited Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PUK^","name":"Prudential Public Limited Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PUK^A","name":"Prudential Public Limited Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PUMP","name":"ProPetro Holding Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PVG","name":"Pretium Resources, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PVH","name":"PVH Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PVL","name":"Permianville Royalty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PVT","name":"Pivotal Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PVTL","name":"Pivotal Software, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PVT.U","name":"Pivotal Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PVT.WS","name":"Pivotal Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PWR","name":"Quanta Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PXD","name":"Pioneer Natural Resources Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PYN","name":"PIMCO New York Municipal Income Fund III","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PYS","name":"PPlus Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PYT","name":"PPlus Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PYX","name":"Pyxus International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PZC","name":"PIMCO California Municipal Income Fund III","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"PZN","name":"Pzena Investment Management Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"QD","name":"Qudian Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"QEP","name":"QEP Resources, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"QES","name":"Quintana Energy Services Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"QGEN","name":"Qiagen N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"QHC","name":"Quorum Health Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"QSR","name":"Restaurant Brands International Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"QTS","name":"QTS Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"QTS^A","name":"QTS Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"QTS^B","name":"QTS Realty Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"QTWO","name":"Q2 Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"QUAD","name":"Quad Graphics, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"QUOT","name":"Quotient Technology Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"QVCD","name":"QVC, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"R","name":"Ryder System, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RA","name":"Brookfield Real Assets Income Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RACE","name":"Ferrari N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RAD","name":"Rite Aid Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RAMP","name":"LiveRamp Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RBA","name":"Ritchie Bros. Auctioneers Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RBC","name":"Regal Beloit Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RBS","name":"Royal Bank Scotland plc (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RC","name":"Ready Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RCA","name":"Ready Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RCB","name":"Ready Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RCI","name":"Rogers Communication, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RCL","name":"Royal Caribbean Cruises Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RCP","name":"Ready Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RCS","name":"PIMCO Strategic Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RCUS","name":"Arcus Biosciences, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RDN","name":"Radian Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RDS.A","name":"Royal Dutch Shell PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RDS.B","name":"Royal Dutch Shell PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RDY","name":"Dr. Reddy&#39;s Laboratories Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RE","name":"Everest Re Group, Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RELX","name":"RELX PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RENN","name":"Renren Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RES","name":"RPC, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RESI","name":"Front Yard Residential Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"REV","name":"Revlon, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"REVG","name":"REV Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"REX","name":"REX American Resources Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"REXR","name":"Rexford Industrial Realty, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"REXR^A","name":"Rexford Industrial Realty, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"REXR^B","name":"Rexford Industrial Realty, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"REZI","name":"Resideo Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RF","name":"Regions Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RF^A","name":"Regions Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RF^B","name":"Regions Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RF^C","name":"Regions Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RFI","name":"Cohen & Steers Total Return Realty Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RFP","name":"Resolute Forest Products Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RGA","name":"Reinsurance Group of America, Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RGR","name":"Sturm, Ruger & Company, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RGS","name":"Regis Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RGT","name":"Royce Global Value Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RH","name":"RH","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RHI","name":"Robert Half International Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RHP","name":"Ryman Hospitality Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RIG","name":"Transocean Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RIO","name":"Rio Tinto Plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RIV","name":"RiverNorth Opportunities Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RJF","name":"Raymond James Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RL","name":"Ralph Lauren Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RLGY","name":"Realogy Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RLH","name":"Red Lion Hotels Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RLI","name":"RLI Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RLJ","name":"RLJ Lodging Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RLJ^A","name":"RLJ Lodging Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RM","name":"Regional Management Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RMAX","name":"RE/MAX Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RMD","name":"ResMed Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RMED","name":"Ra Medical Systems, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RMG","name":"RMG Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RMG.U","name":"RMG Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RMG.WS","name":"RMG Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RMI","name":"RiverNorth Opportunistic Municipal Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RMM","name":"RiverNorth Managed Duration Municipal Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RMPL^","name":"RiverNorth Marketplace Lending Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RMT","name":"Royce Micro-Cap Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RNG","name":"Ringcentral, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RNGR","name":"Ranger Energy Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RNP","name":"Cohen & Steers REIT and Preferred and Income Fund,","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RNR","name":"RenaissanceRe Holdings Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RNR^C","name":"RenaissanceRe Holdings Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RNR^E","name":"RenaissanceRe Holdings Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RNR^F","name":"RenaissanceRe Holdings Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ROAN","name":"Roan Resources, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ROG","name":"Rogers Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ROK","name":"Rockwell Automation, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ROL","name":"Rollins, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ROP","name":"Roper Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ROYT","name":"Pacific Coast Oil Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RPAI","name":"Retail Properties of America, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RPLA","name":"Replay Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RPLA.U","name":"Replay Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RPLA.WS","name":"Replay Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RPM","name":"RPM International Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RPT","name":"RPT Realty","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RPT^D","name":"RPT Realty","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RQI","name":"Cohen & Steers Quality Income Realty Fund Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RRC","name":"Range Resources Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RRD","name":"R.R. Donnelley & Sons Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RRTS","name":"Roadrunner Transportation Systems, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RS","name":"Reliance Steel & Aluminum Co.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RSF","name":"RiverNorth Marketplace Lending Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RSG","name":"Republic Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RST","name":"Rosetta Stone","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RTEC","name":"Rudolph Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RTN","name":"Raytheon Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RTW","name":"RTW Retailwinds, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RUBI","name":"The Rubicon Project, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RVI","name":"Retail Value Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RVLV","name":"Revolve Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RVT","name":"Royce Value Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RWGE","name":"Regalwood Global Energy Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RWGE.U","name":"Regalwood Global Energy Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RWGE.WS","name":"Regalwood Global Energy Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RWT","name":"Redwood Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RXN","name":"Rexnord Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RXN^A","name":"Rexnord Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RY","name":"Royal Bank Of Canada","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RYAM","name":"Rayonier Advanced Materials Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RYB","name":"RYB Education, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RYI","name":"Ryerson Holding Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RYN","name":"Rayonier Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RY^T","name":"Royal Bank Of Canada","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RZA","name":"Reinsurance Group of America, Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"RZB","name":"Reinsurance Group of America, Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"S","name":"Sprint Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SA","name":"Seabridge Gold, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SAB","name":"Saratoga Investment Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SAF","name":"Saratoga Investment Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SAFE","name":"Safehold Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SAH","name":"Sonic Automotive, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SAIC","name":"SCIENCE APPLICATIONS INTERNATIONAL CORPORATION","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SAIL","name":"SailPoint Technologies Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SALT","name":"Scorpio Bulkers Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SAM","name":"Boston Beer Company, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SAN","name":"Banco Santander, S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SAN^B","name":"Banco Santander, S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SAP","name":"SAP SE","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SAR","name":"Saratoga Investment Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SAVE","name":"Spirit Airlines, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SB","name":"Safe Bulkers, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SB^C","name":"Safe Bulkers, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SB^D","name":"Safe Bulkers, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SBE.U","name":"Switchback Energy Acquisition Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SBGL","name":"Sibanye Gold Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SBH","name":"Sally Beauty Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SBI","name":"Western Asset Intermediate Muni Fund Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SBNA","name":"Scorpio Tankers Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SBOW","name":"SilverBow Resorces, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SBR","name":"Sabine Royalty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SBS","name":"Companhia de saneamento Basico Do Estado De Sao Paulo - Sabesp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SC","name":"Santander Consumer USA Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCA","name":"Stellus Capital Investment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCCO","name":"Southern Copper Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCD","name":"LMP Capital and Income Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCE^G","name":"Southern California Edison Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCE^H","name":"Southern California Edison Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCE^J","name":"Southern California Edison Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCE^K","name":"Southern California Edison Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCE^L","name":"Southern California Edison Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCHW","name":"The Charles Schwab Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCHW^C","name":"The Charles Schwab Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCHW^D","name":"The Charles Schwab Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCI","name":"Service Corporation International","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCL","name":"Stepan Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCM","name":"Stellus Capital Investment Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCPE","name":"SC Health Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCPE.U","name":"SC Health Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCPE.WS","name":"SC Health Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCS","name":"Steelcase Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SCX","name":"L.S. Starrett Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SD","name":"SandRidge Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SDR","name":"SandRidge Mississippian Trust II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SDRL","name":"Seadrill Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SDT","name":"SandRidge Mississippian Trust I","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SE","name":"Sea Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SEAS","name":"SeaWorld Entertainment, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SEE","name":"Sealed Air Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SEM","name":"Select Medical Holdings Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SEMG","name":"Semgroup Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SERV","name":"ServiceMaster Global Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SF","name":"Stifel Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SF^A","name":"Stifel Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SFB","name":"Stifel Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SF^B","name":"Stifel Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SFE","name":"Safeguard Scientifics, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SFL","name":"Ship Finance International Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SFUN","name":"Fang Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SGU","name":"Star Group, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SHAK","name":"Shake Shack, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SHG","name":"Shinhan Financial Group Co Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SHI","name":"SINOPEC Shangai Petrochemical Company, Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SHLL","name":"Tortoise Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SHLL.U","name":"Tortoise Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SHLL.WS","name":"Tortoise Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SHLX","name":"Shell Midstream Partners, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SHO","name":"Sunstone Hotel Investors, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SHO^E","name":"Sunstone Hotel Investors, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SHO^F","name":"Sunstone Hotel Investors, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SHOP","name":"Shopify Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SHW","name":"Sherwin-Williams Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SID","name":"National Steel Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SIG","name":"Signet Jewelers Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SITC","name":"SITE Centers Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SITC^A","name":"SITE Centers Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SITC^J","name":"SITE Centers Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SITC^K","name":"SITE Centers Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SITE","name":"SiteOne Landscape Supply, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SIX","name":"Six Flags Entertainment Corporation New","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SJI","name":"South Jersey Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SJIU","name":"South Jersey Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SJM","name":"J.M. Smucker Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SJR","name":"Shaw Communications Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SJT","name":"San Juan Basin Royalty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SJW","name":"SJW Group","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SKM","name":"SK Telecom Co., Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SKT","name":"Tanger Factory Outlet Centers, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SKX","name":"Skechers U.S.A., Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SKY","name":"Skyline Champion Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SLB","name":"Schlumberger N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SLCA","name":"U.S. Silica Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SLF","name":"Sun Life Financial Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SLG","name":"SL Green Realty Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SLG^I","name":"SL Green Realty Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SM","name":"SM Energy Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SMAR","name":"Smartsheet Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SMFG","name":"Sumitomo Mitsui Financial Group Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SMG","name":"Scotts Miracle-Gro Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SMHI","name":"SEACOR Marine Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SMLP","name":"Summit Midstream Partners, LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SMM","name":"Salient Midstream & MLP Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SMP","name":"Standard Motor Products, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SMTA","name":"Spirit MTA REIT","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SNA","name":"Snap-On Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SNAP","name":"Snap Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SNDR","name":"Schneider National, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SNE","name":"Sony Corp Ord","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SNN","name":"Smith & Nephew SNATS, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SNP","name":"China Petroleum & Chemical Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SNR","name":"New Senior Investment Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SNV","name":"Synovus Financial Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SNV^D","name":"Synovus Financial Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SNV^E","name":"Synovus Financial Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SNX","name":"Synnex Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SO","name":"Southern Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SOGO","name":"Sogou Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SOI","name":"Solaris Oilfield Infrastructure, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SOJA","name":"Southern Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SOJB","name":"Southern Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SOJC","name":"Southern Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SOL","name":"Renesola Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SOLN","name":"Southern Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SON","name":"Sonoco Products Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SOR","name":"Source Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPAQ","name":"Spartan Energy Acquisition Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPAQ.U","name":"Spartan Energy Acquisition Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPAQ.WS","name":"Spartan Energy Acquisition Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPB           ","name":"Spectrum Brands Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPE","name":"Special Opportunities Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPE^B","name":"Special Opportunities Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPG","name":"Simon Property Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPGI","name":"S&P Global Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPG^J","name":"Simon Property Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPH","name":"Suburban Propane Partners, L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPLP","name":"Steel Partners Holdings LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPLP^A","name":"Steel Partners Holdings LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPN","name":"Superior Energy Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPOT","name":"Spotify Technology S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPR","name":"Spirit Aerosystems Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPXC","name":"SPX Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SPXX","name":"Nuveen S&P 500 Dynamic Overwrite Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SQ","name":"Square, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SQM","name":"Sociedad Quimica y Minera S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SQNS","name":"Sequans Communications S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SR","name":"Spire Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SR^A","name":"Spire Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SRC","name":"Spirit Realty Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SRC^A","name":"Spirit Realty Capital, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SRE","name":"Sempra Energy","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SREA","name":"Sempra Energy","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SRE^A","name":"Sempra Energy","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SRE^B","name":"Sempra Energy","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SRF","name":"Cushing Energy Income Fund (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SRG","name":"Seritage Growth Properties","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SRG^A","name":"Seritage Growth Properties","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SRI","name":"Stoneridge, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SRL","name":"Scully Royalty Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SRLP","name":"Sprague Resources LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SRT","name":"StarTek, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SRV","name":"Cushing MLP & Infrastructure Total Return Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SSD","name":"Simpson Manufacturing Company, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SSI","name":"Stage Stores, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SSL","name":"Sasol Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SSTK","name":"Shutterstock, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SSW","name":"Seaspan Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SSWA","name":"Seaspan Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SSW^D","name":"Seaspan Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SSW^E","name":"Seaspan Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SSW^G","name":"Seaspan Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SSW^H","name":"Seaspan Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SSW^I","name":"Seaspan Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ST","name":"Sensata Technologies Holding plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STAG","name":"Stag Industrial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STAG^C","name":"Stag Industrial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STAR          ","name":"iStar Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STAR^D","name":"iStar Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STAR^G","name":"iStar Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STAR^I","name":"iStar Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STC","name":"Stewart Information Services Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STE","name":"STERIS plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STG","name":"Sunlands Technology Group","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STI","name":"SunTrust Banks, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STI^A","name":"SunTrust Banks, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STK","name":"Columbia Seligman Premium Technology Growth Fund, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STL","name":"Sterling Bancorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STL^A","name":"Sterling Bancorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STM","name":"STMicroelectronics N.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STN","name":"Stantec Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STNG","name":"Scorpio Tankers Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STON","name":"StoneMor Partners L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STOR","name":"STORE Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STT","name":"State Street Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STT^C","name":"State Street Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STT^D","name":"State Street Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STT^E","name":"State Street Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STT^G","name":"State Street Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STWD","name":"STARWOOD PROPERTY TRUST, INC.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STZ","name":"Constellation Brands Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"STZ.B","name":"Constellation Brands Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SU","name":"Suncor Energy  Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SUI","name":"Sun Communities, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SUM","name":"Summit Materials, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SUN","name":"Sunoco LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SUP","name":"Superior Industries International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SUPV","name":"Grupo Supervielle S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SUZ","name":"Suzano S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SWCH","name":"Switch, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SWI","name":"SolarWinds Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SWJ","name":"Stanley Black & Decker, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SWK","name":"Stanley Black & Decker, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SWM","name":"Schweitzer-Mauduit International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SWN","name":"Southwestern Energy Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SWP","name":"Stanley Black & Decker, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SWX","name":"Southwest Gas Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SWZ","name":"Swiss Helvetia Fund, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SXC","name":"SunCoke Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SXI","name":"Standex International Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SXT","name":"Sensient Technologies Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SYF","name":"Synchrony Financial","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SYK","name":"Stryker Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SYX","name":"Systemax Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SYY","name":"Sysco Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"SZC","name":"Cushing Renaissance Fund (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"T","name":"AT&T Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TAC","name":"TransAlta Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TAK","name":"Takeda Pharmaceutical Company Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TAL","name":"TAL Education Group","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TALO","name":"Talos Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TAP","name":"Molson Coors Brewing  Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TAP.A","name":"Molson Coors Brewing  Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TARO","name":"Taro Pharmaceutical Industries Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TBB","name":"AT&T Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TBC","name":"AT&T Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TBI","name":"TrueBlue, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TCI","name":"Transcontinental Realty Investors, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TCO","name":"Taubman Centers, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TCO^J","name":"Taubman Centers, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TCO^K","name":"Taubman Centers, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TCP","name":"TC PipeLines, LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TCRW","name":"THL Credit, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TCRZ","name":"THL Credit, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TCS","name":"Container Store (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TD","name":"Toronto Dominion Bank (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TDA","name":"Telephone and Data Systems, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TDC","name":"Teradata Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TDE","name":"Telephone and Data Systems, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TDF","name":"Templeton Dragon Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TDG","name":"Transdigm Group Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TDI","name":"Telephone and Data Systems, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TDJ","name":"Telephone and Data Systems, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TDOC","name":"Teladoc Health, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TDS","name":"Telephone and Data Systems, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TDW","name":"Tidewater Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TDW.WS.A","name":"Tidewater Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TDW.WS.B","name":"Tidewater Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TDY","name":"Teledyne Technologies Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TEAF","name":"Tortoise Essential Assets Income Term Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TECK","name":"Teck Resources Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TEF","name":"Telefonica SA","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TEI","name":"Templeton Emerging Markets Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TEL","name":"TE Connectivity Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TEN","name":"Tenneco Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TEO","name":"Telecom Argentina Stet - France Telecom S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TEVA","name":"Teva Pharmaceutical Industries Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TEX","name":"Terex Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TFX","name":"Teleflex Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TG","name":"Tredegar Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TGE","name":"Tallgrass Energy, LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TGH","name":"Textainer Group Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TGI","name":"Triumph Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TGNA","name":"TEGNA Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TGP","name":"Teekay LNG Partners L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TGP^A","name":"Teekay LNG Partners L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TGP^B","name":"Teekay LNG Partners L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TGS","name":"Transportadora De Gas Sa Ord B","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TGT","name":"Target Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"THC","name":"Tenet Healthcare Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"THG","name":"The Hanover Insurance Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"THGA","name":"The Hanover Insurance Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"THO","name":"Thor Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"THQ","name":"Tekla Healthcare Opportunies Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"THR","name":"Thermon Group Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"THS","name":"Treehouse Foods, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"THW","name":"Tekla World Healthcare Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TIF","name":"Tiffany & Co.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TISI","name":"Team, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TJX","name":"TJX Companies, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TK","name":"Teekay Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TKC","name":"Turkcell Iletisim Hizmetleri AS","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TKR","name":"Timken Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TLI","name":"Western Asset Corporate Loan Fund Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TLK","name":"PT Telekomunikasi Indonesia, Tbk","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TLRA","name":"Telaria, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TLRD","name":"Tailored Brands, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TLYS","name":"Tilly&#39;s, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TM","name":"Toyota Motor Corp Ltd Ord","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TME","name":"Tencent Music Entertainment Group","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TMHC","name":"Taylor Morrison Home Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TMO","name":"Thermo Fisher Scientific Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TMST","name":"TimkenSteel Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TNC","name":"Tennant Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TNET","name":"TriNet Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TNK","name":"Teekay Tankers Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TNP","name":"Tsakos Energy Navigation Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TNP^C","name":"Tsakos Energy Navigation Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TNP^D","name":"Tsakos Energy Navigation Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TNP^E","name":"Tsakos Energy Navigation Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TNP^F","name":"Tsakos Energy Navigation Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TOL","name":"Toll Brothers, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TOO","name":"Teekay Offshore Partners L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TOO^A","name":"Teekay Offshore Partners L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TOO^B","name":"Teekay Offshore Partners L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TOO^E","name":"Teekay Offshore Partners L.P.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TOT","name":"Total S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TOWR","name":"Tower International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TPB","name":"Turning Point Brands, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TPC","name":"Tutor Perini Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TPGH","name":"TPG Pace Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TPGH.U","name":"TPG Pace Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TPGH.WS","name":"TPG Pace Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TPH","name":"TRI Pointe Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TPL","name":"Texas Pacific Land Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TPR","name":"Tapestry, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TPRE","name":"Third Point Reinsurance Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TPVG","name":"TriplePoint Venture Growth BDC Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TPVY","name":"TriplePoint Venture Growth BDC Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TPX","name":"Tempur Sealy International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TPZ","name":"Tortoise Power and Energy Infrastructure Fund, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TR","name":"Tootsie Roll Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRC","name":"Tejon Ranch Co","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRCO","name":"Tribune Media Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TREC","name":"Trecora Resources","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TREX","name":"Trex Company, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRGP","name":"Targa Resources, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRI","name":"Thomson Reuters Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRK","name":"Speedway Motorsports, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRN","name":"Trinity Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRNE","name":"Trine Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRNE.U","name":"Trine Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRNE.WS","name":"Trine Acquisition Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRNO","name":"Terreno Realty Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TROX","name":"Tronox Holdings plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRP","name":"TC Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRQ","name":"Turquoise Hill Resources Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRTN","name":"Triton International Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRTN^A","name":"Triton International Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRTN^B","name":"Triton International Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRTX","name":"TPG RE Finance Trust, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRU","name":"TransUnion","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRV","name":"The Travelers Companies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TRWH","name":"Twin River Worldwide Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TS","name":"Tenaris S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TSE","name":"Trinseo S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TSI","name":"TCW Strategic Income Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TSLF","name":"THL Credit Senior Loan Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TSLX","name":"TPG Specialty Lending, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TSM","name":"Taiwan Semiconductor Manufacturing Company Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TSN","name":"Tyson Foods, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TSQ","name":"Townsquare Media, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TSS","name":"Total System Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TSU","name":"TIM Participacoes S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TTC","name":"Toro Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TTI","name":"Tetra Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TTM","name":"Tata Motors Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TTP","name":"Tortoise Pipeline & Energy Fund, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TU","name":"TELUS Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TUFN","name":"Tufin Software Technologies Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TUP","name":"Tupperware Brands Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TV","name":"Grupo Televisa S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TVC","name":"Tennessee Valley Authority","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TVE","name":"Tennessee Valley Authority","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TWI","name":"Titan International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TWLO","name":"Twilio Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TWN","name":"Taiwan Fund, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TWO","name":"Two Harbors Investments Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TWO^A","name":"Two Harbors Investments Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TWO^B","name":"Two Harbors Investments Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TWO^C","name":"Two Harbors Investments Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TWO^D","name":"Two Harbors Investments Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TWO^E","name":"Two Harbors Investments Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TWTR","name":"Twitter, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TX","name":"Ternium S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TXT","name":"Textron Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TY","name":"Tri Continental Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TY^","name":"Tri Continental Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TYG","name":"Tortoise Energy Infrastructure Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"TYL","name":"Tyler Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UA","name":"Under Armour, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UAA","name":"Under Armour, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UAN","name":"CVR Partners, LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UBA","name":"Urstadt Biddle Properties Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UBER","name":"Uber Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UBP","name":"Urstadt Biddle Properties Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UBP^G","name":"Urstadt Biddle Properties Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UBP^H","name":"Urstadt Biddle Properties Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UBS","name":"UBS AG","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UDR","name":"UDR, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UE","name":"Urban Edge Properties","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UFI","name":"Unifi, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UFS","name":"Domtar Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UGI","name":"UGI Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UGP","name":"Ultrapar Participacoes S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UHS","name":"Universal Health Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UHT","name":"Universal Health Realty Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UI","name":"Ubiquiti Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UIS","name":"Unisys Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UL","name":"Unilever PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UMC","name":"United Microelectronics Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UMH","name":"UMH Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UMH^B","name":"UMH Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UMH^C","name":"UMH Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UMH^D","name":"UMH Properties, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UN","name":"Unilever NV","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UNF","name":"Unifirst Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UNFI","name":"United Natural Foods, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UNH","name":"UnitedHealth Group Incorporated","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UNM","name":"Unum Group","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UNMA","name":"Unum Group","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UNP","name":"Union Pacific Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UNT","name":"Unit Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UNVR","name":"Univar Solutions Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UPS","name":"United Parcel Service, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"URI","name":"United Rentals, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USA","name":"Liberty All-Star Equity Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USAC","name":"USA Compression Partners, LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USB","name":"U.S. Bancorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USB^A","name":"U.S. Bancorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USB^H","name":"U.S. Bancorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USB^M","name":"U.S. Bancorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USB^O","name":"U.S. Bancorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USB^P","name":"U.S. Bancorp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USDP","name":"USD Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USFD","name":"US Foods Holding Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USM","name":"United States Cellular Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USNA","name":"USANA Health Sciences, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USPH","name":"U.S. Physical Therapy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"USX","name":"U.S. Xpress Enterprises, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UTF","name":"Cohen & Steers Infrastructure Fund, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UTI","name":"Universal Technical Institute Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UTL","name":"UNITIL Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UTX","name":"United Technologies Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UVE","name":"UNIVERSAL INSURANCE HOLDINGS INC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UVV","name":"Universal Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UZA","name":"United States Cellular Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UZB","name":"United States Cellular Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"UZC","name":"United States Cellular Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"V","name":"Visa Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VAC","name":"Marriott Vacations Worldwide Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VAL","name":"Valaris plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VALE","name":"VALE S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VAM","name":"The Vivaldi Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VAPO","name":"Vapotherm, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VAR","name":"Varian Medical Systems, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VBF","name":"Invesco Bond Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VCIF","name":"Vertical Capital Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VCRA","name":"Vocera Communications, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VCV","name":"Invesco California Value Municipal Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VEC","name":"Vectrus, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VEDL","name":"Vedanta  Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VEEV","name":"Veeva Systems Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VER","name":"VEREIT Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VER^F","name":"VEREIT Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VET","name":"Vermilion Energy Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VFC","name":"V.F. Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VG","name":"Vonage Holdings Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VGI","name":"Virtus Global Multi-Sector Income Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VGM","name":"Invesco Trust for Investment Grade Municipals","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VGR","name":"Vector Group Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VHI","name":"Valhi, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VICI","name":"VICI Properties Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VIPS","name":"Vipshop Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VIST","name":"Vista Oil & Gas, S.A.B. de C.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VIV","name":"Telefonica Brasil S.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VJET","name":"voxeljet AG","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VKQ","name":"Invesco Municipal Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VLO","name":"Valero Energy Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VLRS","name":"Controladora Vuela Compania de Aviacion, S.A.B. de C.V.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VLT","name":"Invesco High Income Trust II","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VMC","name":"Vulcan Materials Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VMI","name":"Valmont Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VMO","name":"Invesco Municipal Opportunity Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VMW","name":"Vmware, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VNCE","name":"Vince Holding Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VNE","name":"Veoneer, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VNO","name":"Vornado Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VNO^K","name":"Vornado Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VNO^L","name":"Vornado Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VNO^M","name":"Vornado Realty Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VNTR","name":"Venator Materials PLC","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VOC","name":"VOC Energy Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VOYA","name":"Voya Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VOYA^B","name":"Voya Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VPG","name":"Vishay Precision Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VPV","name":"Invesco Pennsylvania Value Municipal Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VRS","name":"Verso Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VRTV","name":"Veritiv Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VSH","name":"Vishay Intertechnology, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VSI","name":"Vitamin Shoppe, Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VSLR","name":"Vivint Solar, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VSM","name":"Versum Materials, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VST","name":"Vistra Energy Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VSTO","name":"Vista Outdoor Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VST.WS.A","name":"Vistra Energy Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VTA","name":"Invesco Credit Opportunities Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VTN","name":"Invesco Trust  for Investment Grade New York Municipal","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VTR","name":"Ventas, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VVI","name":"Viad Corp","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VVR","name":"Invesco Senior Income Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VVV","name":"Valvoline Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"VZ","name":"Verizon Communications Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"W","name":"Wayfair Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WAAS","name":"AquaVenture Holdings Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WAB","name":"Westinghouse Air Brake Technologies Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WAIR","name":"Wesco Aircraft Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WAL","name":"Western Alliance Bancorporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WALA","name":"Western Alliance Bancorporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WAT","name":"Waters Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WBAI","name":"500.com Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WBC","name":"Wabco Holdings Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WBK","name":"Westpac Banking Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WBS","name":"Webster Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WBS^F","name":"Webster Financial Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WBT","name":"Welbilt, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WCC","name":"WESCO International, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WCG","name":"WellCare Health Plans, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WCN","name":"Waste Connections, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WD","name":"Walker & Dunlop, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WDR","name":"Waddell & Reed Financial, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WEA","name":"Western Asset Bond Fund","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WEC","name":"WEC Energy Group, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WEI","name":"Weidai Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WELL","name":"Welltower Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WES","name":"Western Midstream Partners, LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WEX","name":"WEX Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WF","name":"Woori Bank","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WFC","name":"Wells Fargo & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WFC^L","name":"Wells Fargo & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WFC^N","name":"Wells Fargo & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WFC^O","name":"Wells Fargo & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WFC^P","name":"Wells Fargo & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WFC^Q","name":"Wells Fargo & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WFC^R","name":"Wells Fargo & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WFC^T","name":"Wells Fargo & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WFC^V","name":"Wells Fargo & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WFC^W","name":"Wells Fargo & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WFC^X","name":"Wells Fargo & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WFC^Y","name":"Wells Fargo & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WFE^A","name":"Wells Fargo & Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WGO","name":"Winnebago Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WH","name":"Wyndham Hotels & Resorts, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WHD","name":"Cactus, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WHG","name":"Westwood Holdings Group Inc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WHR","name":"Whirlpool Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WIA","name":"Western Asset/Claymore U.S. Treasury Inflation Prot Secs Fd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WIT","name":"Wipro Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WIW","name":"Western Asset/Claymore U.S Treasury Inflation Prot Secs Fd 2","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WK","name":"Workiva Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WLH","name":"Lyon William Homes","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WLK","name":"Westlake Chemical Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WLKP","name":"Westlake Chemical Partners LP","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WLL","name":"Whiting Petroleum Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WM","name":"Waste Management, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WMB","name":"Williams Companies, Inc. (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WMC","name":"Western Asset Mortgage Capital Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WMK","name":"Weis Markets, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WMS","name":"Advanced Drainage Systems, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WMT","name":"Walmart Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WNC","name":"Wabash National Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WNS","name":"WNS (Holdings) Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WOR","name":"Worthington Industries, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WORK","name":"Slack Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WOW","name":"WideOpenWest, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WPC","name":"W.P. Carey Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WPG","name":"Washington Prime Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WPG^H","name":"Washington Prime Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WPG^I","name":"Washington Prime Group Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WPM","name":"Wheaton Precious Metals Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WPP","name":"WPP plc","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WPX","name":"WPX Energy, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WRB","name":"W.R. Berkley Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WRB^B","name":"W.R. Berkley Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WRB^C","name":"W.R. Berkley Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WRB^D","name":"W.R. Berkley Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WRB^E","name":"W.R. Berkley Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WRE","name":"Washington Real Estate Investment Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WRI","name":"Weingarten Realty Investors","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WRK","name":"Westrock Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WSM","name":"Williams-Sonoma, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WSO","name":"Watsco, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WSO.B","name":"Watsco, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WSR","name":"Whitestone REIT","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WST","name":"West Pharmaceutical Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WTI","name":"W&T Offshore, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WTM","name":"White Mountains Insurance Group, Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WTR","name":"Aqua America, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WTRU","name":"Aqua America, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WTS","name":"Watts Water Technologies, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WTTR","name":"Select Energy Services, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WU","name":"Western Union Company (The)","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WUBA","name":"58.com Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WWE","name":"World Wrestling Entertainment, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WWW","name":"Wolverine World Wide, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WY","name":"Weyerhaeuser Company","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"WYND","name":"Wyndham Destinations, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"X","name":"United States Steel Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"XAN","name":"Exantas Capital Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"XAN^C","name":"Exantas Capital Corp.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"XEC","name":"Cimarex Energy Co","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"XFLT","name":"XAI Octagon Floating Rate & Alternative Income Term Trust","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"XHR","name":"Xenia Hotels & Resorts, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"XIN","name":"Xinyuan Real Estate Co Ltd","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"XOM","name":"Exxon Mobil Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"XPO","name":"XPO Logistics, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"XRF","name":"China Rapid Finance Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"XRX","name":"Xerox Holdings Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"XYF","name":"X Financial","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"XYL","name":"Xylem Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"Y","name":"Alleghany Corporation","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"YELP","name":"Yelp Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"YETI","name":"YETI Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"YEXT","name":"Yext, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"YPF","name":"YPF Sociedad Anonima","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"YRD","name":"Yirendai Ltd.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"YUM","name":"Yum! Brands, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"YUMC","name":"Yum China Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZAYO","name":"Zayo Group Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZB^A","name":"Zions Bancorporation N.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZB^G","name":"Zions Bancorporation N.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZBH","name":"Zimmer Biomet Holdings, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZB^H","name":"Zions Bancorporation N.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZBK","name":"Zions Bancorporation N.A.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZEN","name":"Zendesk, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZF","name":"Virtus Total Return Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZNH","name":"China Southern Airlines Company Limited","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZTO","name":"ZTO Express (Cayman) Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZTR","name":"Virtus Global Dividend & Income Fund Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZTS","name":"Zoetis Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZUO","name":"Zuora, Inc.","is_etf":np.nan,"exchange":"NYSE"},{"ticker":"ZYME","name":"Zymeworks Inc.","is_etf":np.nan,"exchange":"NYSE"}]

In [24]:
all_stocks = pd.read_csv('nasdaqlisted.txt', sep='|')
all_stocks

,Symbol,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
0,AACG,ATA Creativity Global - American Depositary Sh...,G,N,D,100.0,N,N
1,AAL,"American Airlines Group, Inc. - Common Stock",Q,N,N,100.0,N,N
2,AAME,Atlantic American Corporation - Common Stock,G,N,N,100.0,N,N
3,AAOI,"Applied Optoelectronics, Inc. - Common Stock",G,N,N,100.0,N,N
4,AAON,"AAON, Inc. - Common Stock",Q,N,N,100.0,N,N
...,...,...,...,...,...,...,...,...
3540,ZXYZ.A,Nasdaq Symbology Test Common Stock,Q,Y,N,100.0,NaN,N
3541,ZXZZT,NASDAQ TEST STOCK,G,Y,N,100.0,NaN,N
3542,ZYNE,"Zynerba Pharmaceuticals, Inc. - Common Stock",G,N,N,100.0,N,N
3543,ZYXI,"Zynex, Inc. - Common Stock",S,N,N,100.0,N,N


In [28]:
test = Stoinks(all_stocks.Symbol[:-100:300], history=('2y', '1d'))
test.Scores

/home/hackerman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:189: RuntimeWarning: Mean of empty slice.
/home/hackerman/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,AACG,AVCTW,CETXW,DLPNW,FLWS,HHR,KIDS,MRCC,OZK,RESN,SSB,VCLT
Averages20,-0.425,NaN,NaN,NaN,0.76500,-3.64,3.710000,0.4575,-1.576667,-0.1950,1.003333,2.776667
BollingerBands,-0.104,NaN,NaN,NaN,0.59125,-0.59,-0.037273,-0.0320,0.530000,0.0085,-1.716364,2.246250
RSI,NaN,NaN,NaN,NaN,3.42000,NaN,NaN,1.2700,6.430000,0.8500,2.880000,3.610000


In [29]:
test.Optimal

{'AACG': 'BollingerBands',
 'AVCTW': nan,
 'CETXW': nan,
 'DLPNW': nan,
 'FLWS': 'RSI',
 'HHR': 'BollingerBands',
 'KIDS': 'Averages20',
 'MRCC': 'RSI',
 'OZK': 'RSI',
 'RESN': 'RSI',
 'SSB': 'RSI',
 'VCLT': 'RSI'}